### 1st part: include modification of code to have drugId and studyId

In [1]:
#### 08.01.2025
#### ALL PHASES 
from array import ArrayType
from functions import (
    relative_success,
    spreadSheetFormatter,
    discrepancifier,
    temporary_directionOfEffect,
    
)
from stoppedTrials import terminated_td
from DoEAssessment import directionOfEffect
from membraneTargets import target_membrane
from pyspark.sql import SparkSession, Window
import pyspark.sql.functions as F
from datetime import datetime
from datetime import date
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.types import (
    StructType,
    StructField,
    DoubleType,
    DecimalType,
    StringType,
    FloatType,
)
import pandas as pd

spark = SparkSession.builder.getOrCreate()

path = "gs://open-targets-pre-data-releases/24.12-uo_test-3/output/etl/parquet/"

target = spark.read.parquet(f"{path}targets/")

diseases = spark.read.parquet(f"{path}diseases/")

evidences = spark.read.parquet(f"{path}evidence")

credible = spark.read.parquet(f"{path}credibleSet")

### index with new fix" "gs://ot-team/irene/gentropy/study_index_2412_fixed"
index = spark.read.parquet(f"gs://ot-team/irene/gentropy/study_index_2412_fixed")

new = spark.read.parquet(f"{path}colocalisation/coloc")

variantIndex = spark.read.parquet(f"{path}variantIndex")

biosample = spark.read.parquet(f"{path}biosample")


#### Fixing scXQTL as XQTLs:
## code provided by @ireneisdoomed
pd.DataFrame.iteritems = pd.DataFrame.items

raw_studies_metadata_schema: StructType = StructType(
        [
            StructField("study_id", StringType(), True),
            StructField("dataset_id", StringType(), True),
            StructField("study_label", StringType(), True),
            StructField("sample_group", StringType(), True),
            StructField("tissue_id", StringType(), True),
            StructField("tissue_label", StringType(), True),
            StructField("condition_label", StringType(), True),
            StructField("sample_size", IntegerType(), True),
            StructField("quant_method", StringType(), True),
            StructField("pmid", StringType(), True),
            StructField("study_type", StringType(), True),
        ]
    )
raw_studies_metadata_path = "https://raw.githubusercontent.com/eQTL-Catalogue/eQTL-Catalogue-resources/fe3c4b4ed911b3a184271a6aadcd8c8769a66aba/data_tables/dataset_metadata.tsv"

study_table = spark.createDataFrame(
            pd.read_csv(raw_studies_metadata_path, sep="\t"),
            schema=raw_studies_metadata_schema,
        )

#index = spark.read.parquet("gs://open-targets-pre-data-releases/24.12-uo_test-3/output/genetics/parquet/study_index")

study_index_w_correct_type = (
    study_table.select(
        F.concat_ws(
            "_",
            F.col("study_label"),
            F.col("quant_method"),
            F.col("sample_group"),
        ).alias("extracted_column"),
        "study_type",
    )
    .join(
        index
        # Get eQTL Catalogue studies
        .filter(F.col("studyType") != "gwas")
        .filter(~F.col("studyId").startswith("UKB_PPP"))
        # Remove measured trait
        .withColumn(
            "extracted_column",
            F.regexp_replace(F.col("studyId"), r"(_ENS.*|_ILMN.*|_X.*|_[0-9]+:.*)", ""),
        )
        .withColumn(
            "extracted_column",
            # After the previous cleanup, there are multiple traits from the same publication starting with the gene symbol that need to be removed (e.g. `Sun_2018_aptamer_plasma_ANXA2.4961.17.1..1`)
            F.when(
                F.col("extracted_column").startswith("Sun_2018_aptamer_plasma"),
                F.lit("Sun_2018_aptamer_plasma"),
            ).otherwise(F.col("extracted_column")),
        ),
        on="extracted_column",
        how="right",
    )
    .persist()
)

fixed = (
    study_index_w_correct_type.withColumn(
        "toFix",
        F.when(
            (F.col("study_type") != "single-cell")
            & (F.col("studyType").startswith("sc")),
            F.lit(True),
        ).otherwise(F.lit(False)),
    )
    # Remove the substring "sc" from the studyType column
    .withColumn(
        "newStudyType",
        F.when(
            F.col("toFix"), F.regexp_replace(F.col("studyType"), r"sc", "")
        ).otherwise(F.col("studyType")),
    )
    .drop("toFix", "extracted_column", "study_type")
).persist()
all_studies = index.join(
    fixed.selectExpr("studyId", "newStudyType"), on="studyId", how="left"
).persist()
fixedIndex = all_studies.withColumn(
    "studyType",
    F.when(F.col("newStudyType").isNotNull(), F.col("newStudyType")).otherwise(
        F.col("studyType")
    ),
).drop("newStudyType")
#### fixed  

newColoc = (
    new.join(
        credible.selectExpr(  #### studyLocusId from credible set to uncover the codified variants on left side
            "studyLocusId as leftStudyLocusId",
            "StudyId as leftStudyId",
            "variantId as leftVariantId",
            "studyType as credibleLeftStudyType",
        ),
        on="leftStudyLocusId",
        how="left",
    )
    .join(
        credible.selectExpr(  #### studyLocusId from credible set to uncover the codified variants on right side
            "studyLocusId as rightStudyLocusId",
            "studyId as rightStudyId",
            "variantId as rightVariantId",
            "studyType as credibleRightStudyType",
        ),
        on="rightStudyLocusId",
        how="left",
    )
    .join(
        fixedIndex.selectExpr(  ### bring modulated target on right side (QTL study)
            "studyId as rightStudyId", "geneId", "projectId", "studyType as indexStudyType", "condition", "biosampleId"
        ),
        on="rightStudyId",
        how="left",
    )
    .persist()
)
# remove columns without content (only null values on them)
df = evidences.filter((F.col("datasourceId") == "gwas_credible_sets"))

# Use an aggregation to determine non-null columns
non_null_counts = df.select(
    *[F.sum(F.col(col).isNotNull().cast("int")).alias(col) for col in df.columns]
)

# Collect the counts for each column
non_null_columns = [
    row[0] for row in non_null_counts.collect()[0].asDict().items() if row[1] > 0
]

# Select only the non-null columns
filtered_df = df.select(*non_null_columns).persist()

## bring studyId, variantId, beta from Gwas and pValue
gwasComplete = filtered_df.join(
    credible.selectExpr(
        "studyLocusId", "studyId", "variantId", "beta as betaGwas", "pValueExponent"
    ),
    on="studyLocusId",
    how="left",
).persist()

resolvedColoc = (
    (
        newColoc.withColumnRenamed("geneId", "targetId")
        .join(
            gwasComplete.withColumnRenamed("studyLocusId", "leftStudyLocusId"),
            on=["leftStudyLocusId", "targetId"],
            how="inner",
        )
        .join(  ### propagated using parent terms
            diseases.selectExpr(
                "id as diseaseId", "name", "parents", "therapeuticAreas"
            ),
            on="diseaseId",
            how="left",
        )
        .withColumn(
            "diseaseId",
            F.explode_outer(F.concat(F.array(F.col("diseaseId")), F.col("parents"))),
        )
        .drop("parents", "oldDiseaseId")
    )
    .withColumn(
        "colocDoE",
        F.when(
            F.col("rightStudyType").isin(
                ["eqtl", "pqtl", "tuqtl", "sceqtl", "sctuqtl"]
            ),
            F.when(
                (F.col("betaGwas") > 0) & (F.col("betaRatioSignAverage") > 0),
                F.lit("GoF_risk"),
            )
            .when(
                (F.col("betaGwas") > 0) & (F.col("betaRatioSignAverage") < 0),
                F.lit("LoF_risk"),
            )
            .when(
                (F.col("betaGwas") < 0) & (F.col("betaRatioSignAverage") > 0),
                F.lit("LoF_protect"),
            )
            .when(
                (F.col("betaGwas") < 0) & (F.col("betaRatioSignAverage") < 0),
                F.lit("GoF_protect"),
            ),
        ).when(
            F.col("rightStudyType").isin(
                ["sqtl", "scsqtl"]
            ),  ### opposite directionality than sqtl
            F.when(
                (F.col("betaGwas") > 0) & (F.col("betaRatioSignAverage") > 0),
                F.lit("LoF_risk"),
            )
            .when(
                (F.col("betaGwas") > 0) & (F.col("betaRatioSignAverage") < 0),
                F.lit("GoF_risk"),
            )
            .when(
                (F.col("betaGwas") < 0) & (F.col("betaRatioSignAverage") > 0),
                F.lit("GoF_protect"),
            )
            .when(
                (F.col("betaGwas") < 0) & (F.col("betaRatioSignAverage") < 0),
                F.lit("LoF_protect"),
            ),
        ),
    )
    .persist()
)

path = "gs://open-targets-pre-data-releases/24.12-uo_test-3/output/etl/parquet/"

datasource_filter = [
    "ot_genetics_portal",
    "gwas_credible_sets",
    "gene_burden",
    "eva",
    "eva_somatic",
    "gene2phenotype",
    "orphanet",
    "cancer_gene_census",
    "intogen",
    "impc",
    "chembl",
]

assessment, evidences, actionType, oncolabel = temporary_directionOfEffect(
    path, datasource_filter
)

drugApproved=spark.read.parquet("gs://ot-team/irene/l2g/validation/chembl_w_flags").drop("clinicalTrialId","isComplex"
).withColumn("isApproved", F.when(F.col("isApproved")=="true", F.lit(1)).otherwise(F.lit(0))).distinct()


spark session created at 2025-01-17 11:20:40.774677
Analysis started on 2025-01-17 at  2025-01-17 11:20:40.774677


25/01/17 11:20:46 WARN package: Truncated the string representation of a plan since it was too large. This behavior can be adjusted by setting 'spark.sql.debug.maxToStringFields'.


#### 2nd part

In [2]:
### include drugId and studyId becuase there are some studies where there are multiple drugs. 
# Everywthing will become coherent in chembl assoc since this
### 1nd version of 

analysis_chembl_indication = (
    discrepancifier(
        assessment.filter(
            (F.col("datasourceId") == "chembl")
            & (F.col("homogenized") != "noEvaluable")
        )
        .join(drugApproved.filter(F.col("isApproved")==1), on=["targetId","diseaseId","drugId"], how="left")
        .withColumn(
            "maxClinPhase", ### no longer maxclinphase for T-D
            F.max("clinicalPhase").over(Window.partitionBy("targetId", "diseaseId","drugId", "studyId","clinicalPhase")), ### this is key to understand what we do
        )
        .withColumn(
        "approvedDrug",
        F.max(F.col("isApproved")).over(Window.partitionBy("targetId", "diseaseId")),
        )
        ## do not consider drugId so when there are two drugs with diferent MoA at the same clinical phase we will not include it (coherency==coherent)
        .groupBy("targetId", "diseaseId", "studyId","drugId","clinicalPhase","maxClinPhase","approvedDrug")  ## we can remove drugID
        .pivot("homogenized")
        .count()
    )
    .withColumnRenamed("studyId","clinicalStudyId")
    .filter(F.col("coherencyDiagonal") == "coherent")
    .drop(
        "coherencyDiagonal", 
        "coherencyOneCell", 
        "noEvaluable", 
        "GoF_risk", 
        "LoF_risk"
    )
    .withColumnRenamed("GoF_protect", "drugGoF_protect")
    .withColumnRenamed("LoF_protect", "drugLoF_protect")
)

benchmark = (
    (
        resolvedColoc.filter(F.col("betaGwas") < 0)
        .join(  ### select just GWAS giving protection
            analysis_chembl_indication, on=["targetId", "diseaseId"], how="inner"
        )
        .withColumn(
            "AgreeDrug",
            F.when(
                (F.col("drugGoF_protect").isNotNull())
                & (F.col("colocDoE") == "GoF_protect"),
                F.lit("yes"),
            )
            .when(
                (F.col("drugLoF_protect").isNotNull())
                & (F.col("colocDoE") == "LoF_protect"),
                F.lit("yes"),
            )
            .otherwise(F.lit("no")),
        )
    )
    .filter(F.col("name") != "COVID-19")  #### remove COVID-19 associations
).join(biosample.select("biosampleId", "biosampleName"), on="biosampleId", how="left")

### Bring examples for the benchmark: how the numbers changed the statistics between pqtl, for instance. 

In [3]:
def comparisons_df_iterative(disdic,projectId):
    toAnalysis=[(key, value) for key, value in disdic.items() if value == projectId]
    schema = StructType(
        [
            StructField("comparison", StringType(), True),
            StructField("comparisonType", StringType(), True),
        ]
    )

    comparisons = spark.createDataFrame(toAnalysis, schema=schema)
    ### include all the columns as predictor

    predictions = spark.createDataFrame(
        data=[
            ("Phase4", "clinical"),
            ("Phase>=3", "clinical"),
            ("Phase>=2", "clinical"),
            ("Phase>=1", "clinical"),
            #("nPhase4", "clinical"),
            #("nPhase>=3", "clinical"),
            #("nPhase>=2", "clinical"),
            #("nPhase>=1", "clinical"),
            ("approved", "clinical"),

            # ("PhaseT", "clinical"),
        ]
    )
    return comparisons.join(predictions, how="full").collect()

full_data = spark.createDataFrame(
    data=[
        ("yes", "yes"),
        ("yes", "no"),
        ("no", "yes"),
        ("no", "no"),
    ],
    schema=StructType(
        [
            StructField("prediction", StringType(), True),
            StructField("comparison", StringType(), True),
        ]
    ),
)

In [39]:
list(set(group1) - set("maxClinPhase_TD"))

['maxClinPhase_TD', 'targetId', 'diseaseId', 'approvedDrug']

In [ ]:
### check if the tiem where we take the maxclinphase is right or not. 

In [23]:
from pyspark.sql.functions import monotonically_increasing_id


In [4]:
chembl_indication=discrepancifier((assessment.filter(
            (F.col("datasourceId") == "chembl")
            & (F.col("homogenized") != "noEvaluable")
        )
        .join(drugApproved.filter(F.col("isApproved")==1), on=["targetId","diseaseId","drugId"], how="left")
        ).withColumn(
        "approvedDrug",
        F.max(F.col("isApproved")).over(Window.partitionBy("targetId", "diseaseId","drugId")),
        ).groupBy("targetId", "diseaseId", "studyId","drugId","clinicalPhase","approvedDrug"
        ).pivot("homogenized").count()
    ).withColumnRenamed("studyId","clinicalStudyId"
        ).filter(F.col("coherencyDiagonal") == "coherent"
        ).drop(
        "coherencyDiagonal", 
        "coherencyOneCell", 
        "noEvaluable", 
        "GoF_risk", 
        "LoF_risk"
    ).withColumnRenamed("GoF_protect", "drugGoF_protect"
    ).withColumnRenamed("LoF_protect", "drugLoF_protect")

new_benchmark = (
    (
        resolvedColoc.filter(F.col("betaGwas") < 0)
        .join(  ### select just GWAS giving protection
            chembl_indication, on=["targetId", "diseaseId"], how="inner"
        )
        .withColumn(
            "AgreeDrug",
            F.when(
                (F.col("drugGoF_protect").isNotNull())
                & (F.col("colocDoE") == "GoF_protect"),
                F.lit("yes"),
            )
            .when(
                (F.col("drugLoF_protect").isNotNull())
                & (F.col("colocDoE") == "LoF_protect"),
                F.lit("yes"),
            )
            .otherwise(F.lit("no")),
        )
    )
    .filter(F.col("name") != "COVID-19")  #### remove COVID-19 associations
).join(biosample.select("biosampleId", "biosampleName"), on="biosampleId", how="left")

In [9]:
# List of columns to aggregate
numeric_columns = ["0.5", "1.0", "2.0", "3.0", "4.0"]

# Escape column names
escaped_columns = [f"`{c}`" for c in numeric_columns]

# Create aggregation expressions
agg_expressions = [F.sum(F.col(c)).alias(c.strip("`")) for c in escaped_columns]



new_benchmark.groupBy("targetId","diseaseId"
).pivot("clinicalPhase").count().withColumn("id", F.monotonically_increasing_id()
).agg(*agg_expressions).show()

+---+-----+-----+-----+-----+
|0.5|  1.0|  2.0|  3.0|  4.0|
+---+-----+-----+-----+-----+
|771|27407|29184|76940|63229|
+---+-----+-----+-----+-----+



In [ ]:
new_benchmark.select("clinicalPhase","clinicalStudyId").distinct().groupBy("clinicalStudyId").pivot("clinicalPhase").count().agg(*agg_expressions).show()

+---+---+---+----+---+
|0.5|1.0|2.0| 3.0|4.0|
+---+---+---+----+---+
| 13|623|590|1308|863|
+---+---+---+----+---+



In [ ]:
new_benchmark.select("clinicalPhase","clinicalStudyId","rightStudyType"
).distinct().groupBy("rightStudyType"
).pivot("clinicalPhase"
).count().show()

+--------------+----+---+---+----+---+
|rightStudyType| 0.5|1.0|2.0| 3.0|4.0|
+--------------+----+---+---+----+---+
|          sqtl|   2|169|198| 516|309|
|          pqtl|null| 66| 89| 293| 94|
|         tuqtl|   6|199|264| 561|341|
|          eqtl|  13|592|528|1198|825|
|       sctuqtl|   4|101|161| 332|222|
|        sceqtl|   3|129|205| 442|336|
|        scsqtl|null| 42| 54| 141| 72|
+--------------+----+---+---+----+---+



In [ ]:
new_benchmark.filter(F.col("clinicalStudyId").isNotNull()).withColumn("agree_lowestPval", F.first("AgreeDrug").over(window_spec)
).select(
    "targetId",
    "diseaseId",
    "clinicalPhase",
    "clinicalStudyId",
    "rightStudyType",
    "agree_lowestPval").distinct(

    ).groupBy("rightStudyType","agree_lowestPVal"
    ).pivot("clinicalPhase"
    ).count().sort(F.col("rightStudyType").desc()).show()

+--------------+----------------+----+---+---+---+---+
|rightStudyType|agree_lowestPVal| 0.5|1.0|2.0|3.0|4.0|
+--------------+----------------+----+---+---+---+---+
|         tuqtl|             yes|   2|197|207|637|448|
|         tuqtl|              no|   4| 27| 74| 88| 41|
|          sqtl|              no|   2|114|163|442|353|
|          sqtl|             yes|null| 79| 51|230|104|
|       sctuqtl|              no|   2| 48| 53|147|121|
|       sctuqtl|             yes|   2| 53|110|194|101|
|        scsqtl|              no|null|  9|  4|  2|  8|
|        scsqtl|             yes|null| 33| 50|141| 63|
|        sceqtl|             yes|   2| 79|123|321|275|
|        sceqtl|              no|   1| 51| 84|162| 84|
|          pqtl|             yes|null| 39| 64|161| 48|
|          pqtl|              no|null| 27| 27|133| 46|
|          eqtl|             yes|   8|320|324|748|392|
|          eqtl|              no|   5|350|256|853|683|
+--------------+----------------+----+---+---+---+---+



In [15]:
window_spec = Window.partitionBy("targetId", "diseaseId","rightStudyType").orderBy(F.col("pValueExponent").asc())

new_benchmark.filter(F.col("clinicalStudyId").isNotNull()).withColumn("agree_lowestPval", F.first("AgreeDrug").over(window_spec)
).select(
    "targetId",
    "diseaseId",
    "clinicalPhase",
    "clinicalStudyId",
    #"drugId",
    "rightStudyType",
    "agree_lowestPval").distinct(
    ).withColumn(
        "clinicalPhase", ### no longer maxclinphase for T-D
        F.max("clinicalPhase").over(Window.partitionBy("targetId","diseaseId","rightStudyType"))).withColumn(
        "Phase4",
        F.when(F.col("clinicalPhase") == 4, F.lit("yes")).otherwise(F.lit("no"))
        ).withColumn(
        "Phase>=3",
        F.when(F.col("clinicalPhase") >= 3, F.lit("yes")).otherwise(F.lit("no")),
        ).withColumn(
        "Phase>=2",
        F.when(F.col("clinicalPhase") >= 2, F.lit("yes")).otherwise(F.lit("no")),
        ).withColumn(
        "Phase>=1",
        F.when(F.col("clinicalPhase") >= 1, F.lit("yes")).otherwise(F.lit("no")),
        ).groupBy("rightStudyType","agree_lowestPVal"
        ).pivot("Phase4").count().sort(F.col("rightStudyType").desc()
        ).select("rightStudyType","agree_lowestPVal","yes","no").show()

+--------------+----------------+----+---+
|rightStudyType|agree_lowestPVal| yes| no|
+--------------+----------------+----+---+
|         tuqtl|             yes|1114| 99|
|         tuqtl|              no| 456| 56|
|          sqtl|              no| 718| 76|
|          sqtl|             yes| 718| 26|
|       sctuqtl|              no| 353| 19|
|       sctuqtl|             yes| 420| 39|
|        scsqtl|              no|  19|  4|
|        scsqtl|             yes| 286|  1|
|        sceqtl|             yes| 725| 75|
|        sceqtl|              no| 355| 27|
|          pqtl|             yes| 290| 22|
|          pqtl|              no| 212| 21|
|          eqtl|             yes|1609|126|
|          eqtl|              no|2064|140|
+--------------+----------------+----+---+



In [ ]:
window_spec = Window.partitionBy("targetId", "diseaseId","rightStudyType").orderBy(F.col("pValueExponent").asc())

new_benchmark.filter(F.col("clinicalStudyId").isNotNull()).withColumn("agree_lowestPval", F.first("AgreeDrug").over(window_spec)
).select(
    "targetId",
    "diseaseId",
    "clinicalPhase",
    "clinicalStudyId",
    "drugId",
    "rightStudyType",
    "agree_lowestPval").distinct(
    ).withColumn(
        "clinicalPhase", ### no longer maxclinphase for T-D
        F.max("clinicalPhase").over(Window.partitionBy("targetId","diseaseId","rightStudyType"))).withColumn(
        "Phase4",
        F.when(F.col("clinicalPhase") == 4, F.lit("yes")).otherwise(F.lit("no"))
        ).withColumn(
        "Phase>=3",
        F.when(F.col("clinicalPhase") >= 3, F.lit("yes")).otherwise(F.lit("no")),
        ).withColumn(
        "Phase>=2",
        F.when(F.col("clinicalPhase") >= 2, F.lit("yes")).otherwise(F.lit("no")),
        ).withColumn(
        "Phase>=1",
        F.when(F.col("clinicalPhase") >= 1, F.lit("yes")).otherwise(F.lit("no")),
        ).groupBy("rightStudyType","agree_lowestPVal"
        ).pivot("Phase4").count().sort(F.col("rightStudyType").desc()
        ).select("rightStudyType","agree_lowestPVal","yes","no").show()

In [51]:
new_benchmark.select("targetId","diseaseId","clinicalPhase","clinicalStudyId","rightStudyType"
).distinct().groupBy("rightStudyType"
).pivot("clinicalPhase"
).count().show()

+--------------+----+---+---+----+----+
|rightStudyType| 0.5|1.0|2.0| 3.0| 4.0|
+--------------+----+---+---+----+----+
|          sqtl|   2|193|214| 675| 478|
|          pqtl|null| 66| 91| 296| 115|
|         tuqtl|   6|224|282| 730| 517|
|          eqtl|  13|670|582|1610|1126|
|       sctuqtl|   4|101|163| 343| 235|
|        sceqtl|   3|130|207| 486| 378|
|        scsqtl|null| 42| 54| 143|  76|
+--------------+----+---+---+----+----+



In [47]:
new_benchmark.select("clinicalStudyId").distinct().count()

3395

In [24]:
new_benchmark.withColumn("approved_l", F.when(F.col("approvedDrug")==1, F.lit("yes")).otherwise(F.lit("no"))).groupBy("approvedDrug","approved_l").count().show()

+------------+----------+------+
|approvedDrug|approved_l| count|
+------------+----------+------+
|        null|        no| 40787|
|           1|       yes|156744|
+------------+----------+------+



In [39]:
new_benchmark.withColumn(
"approved", ### no longer maxclinphase for T-D
F.max("approvedDrug").over(Window.partitionBy("targetId","diseaseId")
)).withColumn("approved2",F.when(F.col("approved")==1, F.lit("yes")).otherwise(F.lit("no"))
).groupBy("targetId","diseaseId","approved","approved2").agg(F.size(F.collect_set("approved")).alias("size")).sort(F.col("approved").desc()).show()

+---------------+-------------+--------+---------+----+
|       targetId|    diseaseId|approved|approved2|size|
+---------------+-------------+--------+---------+----+
|ENSG00000105397|  EFO_0000540|       1|      yes|   1|
|ENSG00000132170|  EFO_0000400|       1|      yes|   1|
|ENSG00000105397|  EFO_0000676|       1|      yes|   1|
|ENSG00000110944|  EFO_0000676|       1|      yes|   1|
|ENSG00000110944|  EFO_1001494|       1|      yes|   1|
|ENSG00000010310|  EFO_0000400|       1|      yes|   1|
|ENSG00000112164|  EFO_0000400|       1|      yes|   1|
|ENSG00000043591|  EFO_0000319|       1|      yes|   1|
|ENSG00000112164|MONDO_0005148|       1|      yes|   1|
|ENSG00000065989|  EFO_0000676|       1|      yes|   1|
|ENSG00000113161|  EFO_0000400|       1|      yes|   1|
|ENSG00000006071|MONDO_0005148|       1|      yes|   1|
|ENSG00000113161|MONDO_0005148|       1|      yes|   1|
|ENSG00000105397|  EFO_0003778|       1|      yes|   1|
|ENSG00000113302|  EFO_0000540|       1|      ye

In [31]:
new_benchmark.withColumn("approved_l", F.when(F.col("approvedDrug")==1, F.lit("yes")).otherwise(F.lit("no"))).groupBy("targetId","diseaseId").agg(F.size(F.collect_set("approved_l")).alias("size")).sort(F.col("size").desc()).show()

+---------------+-------------+----+
|       targetId|    diseaseId|size|
+---------------+-------------+----+
|ENSG00000105397|  EFO_0000540|   2|
|ENSG00000157388|  EFO_0000319|   2|
|ENSG00000105397|  EFO_0000676|   2|
|ENSG00000113161|MONDO_0005148|   2|
|ENSG00000145777|MONDO_0004979|   2|
|ENSG00000113302|  EFO_0000540|   2|
|ENSG00000065989|  EFO_0000676|   2|
|ENSG00000113302|  EFO_0000676|   2|
|ENSG00000006071|MONDO_0005148|   2|
|ENSG00000113302|  EFO_1001494|   2|
|ENSG00000105397|  EFO_0003778|   2|
|ENSG00000115232|  EFO_0000384|   2|
|ENSG00000110944|  EFO_1001494|   2|
|ENSG00000126218|  EFO_0003827|   2|
|ENSG00000112164|MONDO_0005148|   2|
|ENSG00000126218|  EFO_0004286|   2|
|ENSG00000043591|  EFO_0000319|   2|
|ENSG00000110944|  EFO_0000676|   2|
|ENSG00000112164|  EFO_0000400|   2|
|ENSG00000113161|  EFO_0000400|   2|
+---------------+-------------+----+
only showing top 20 rows



In [43]:
df.to_csv("gs://ot-team/jroldan/analysis/globalqtlResults.csv")

In [47]:
import numpy as np
def convertTuple(tup):
    st = ",".join(map(str, tup))
    return st
from scipy.stats import fisher_exact
from scipy.stats.contingency import odds_ratio
from pyspark.sql.types import *

#### columsn to groupBy - introduce a dictionary for trying different list 
partitionByPValue=["targetId", "diseaseId", "rightStudyType"]

#### description of groups to make groupBy
## partitionForPhase
groupPhase1 = ["targetId","diseaseId"]
groupPhase2 = ["targetId","diseaseId","clinicalPhase"]
groupPhase3 = ["targetId","diseaseId","clinicalStudyId"] 
groupPhase4 = ["targetId","diseaseId","clinicalPhase","clinicalStudyId"] ## will take each clinical phase and study
groupPhase5 = ["targetId","diseaseId","clinicalPhase","clinicalStudyId","drugId"] ## will take max clin phase for DrugId, clinical phase and clnical study
groupPhase6 = ["targetId","diseaseId","clinicalPhase","drugId"] ## will take each max clinical phase per drugId
groupPhase7 = ["targetId","diseaseId","drugId"]

group_phases = {
        "targetIdDiseaseID": groupPhase1,
    "clinicalPhase": groupPhase2,
    "clinicalStudy": groupPhase3,
    "clinicalPhase&ClinicalStudy": groupPhase4,
    "clinicalPhaseClinicalStudyIdDrugId": groupPhase5,
    "clinicalPhaseDrugId": groupPhase6,
    "drugId": groupPhase7,

}

## partitionForRows
groupPhase1 = ["targetId","diseaseId","clinicalPhase"]
groupPhase2 = ["targetId","diseaseId","clinicalPhase","clinicalStudyId"] ## will take each clinical phase and study
groupPhase3 = ["targetId","diseaseId","clinicalPhase","clinicalStudyId","drugId"] ## will take max clin phase for DrugId, clinical phase and clnical study
groupPhase4 = ["targetId","diseaseId","clinicalPhase","drugId"] ## will take each max clinical phase per drugId
# same groups but containing approved 
groupPhase5 = ["targetId","diseaseId","clinicalPhase","approved_l"]
groupPhase6 = ["targetId","diseaseId","clinicalPhase","clinicalStudyId","approved_l"] ## will take each clinical phase and study
groupPhase7 = ["targetId","diseaseId","clinicalPhase","clinicalStudyId","drugId","approved_l"] ## will take max clin phase for DrugId, clinical phase and clnical study
groupPhase8 = ["targetId","diseaseId","clinicalPhase","drugId","approved_l"] ## will take each max clinical phase per drugId

group_rows = {
    "clinicalPhase": groupPhase1,
    "clinicalPhaseClinicalStudyId": groupPhase2,
    "clinicalPhaseClinicalStudyIdDrugId": groupPhase3,
    "clinicalPhaseDrugId": groupPhase4,
    "clinicalPhaseApproved_l": groupPhase5,
    "clinicalPhaseClinicalStudyIdApproved_l": groupPhase6,
    "clinicalPhaseClinicalStudyIdDrugIdApproved_l": groupPhase7,
    "clinicalPhaseDrugIdApproved_l": groupPhase8,

}

value_analysis=["pqtl","eqtl","sqtl"]
### define the window to order for taking pValue 
window_spec = Window.partitionBy(*partitionByPValue).orderBy(F.col("pValueExponent").asc())
results=[]

for value in value_analysis:
    # Iterate over group mapping
    for group_phase, group_phase_columns in group_phases.items():
        print("making group phases:", group_phase_columns)
        for group_row, group_rows_columns in group_rows.items():
                print("making group rows:", group_rows_columns)
                x = value
                print(value,x)

                if "approved_l" in group_rows_columns: 
                        
                        pre=new_benchmark.withColumn("approved_l", F.when(F.col("approvedDrug")==1, F.lit("yes")).otherwise(F.lit("no"))
                                ).filter(F.col("clinicalStudyId").isNotNull()
                                ).withColumn("agree_lowestPval", F.first("AgreeDrug").over(window_spec)
                                ).withColumn(
                                "clinicalPhase", ### no longer maxclinphase for T-D
                                F.max("clinicalPhase").over(Window.partitionBy(*group_phase_columns))
                                ).withColumn(
                                "approved", ### no longer maxclinphase for T-D
                                F.max("approvedDrug").over(Window.partitionBy(*group_phase_columns))
                                ).groupBy(*group_rows_columns).pivot("rightStudyType").agg(F.collect_set("agree_lowestPVal")
                                ).withColumn(
                                        "Phase4",
                                        F.when(F.col("clinicalPhase") == 4, F.lit("yes")).otherwise(F.lit("no"))
                                ).withColumn(
                                        "Phase>=3",
                                        F.when(F.col("clinicalPhase") >= 3, F.lit("yes")).otherwise(F.lit("no")),
                                ).withColumn(
                                        "Phase>=2",
                                        F.when(F.col("clinicalPhase") >= 2, F.lit("yes")).otherwise(F.lit("no")),
                                ).withColumn(
                                        "Phase>=1",
                                        F.when(F.col("clinicalPhase") >= 1, F.lit("yes")).otherwise(F.lit("no")),
                                ).withColumn(
                                        "approved",
                                                F.when(F.col("approved")==1, F.lit("yes")).otherwise(F.lit("no")),
                                ).withColumn(
                                        x, 
                                        F.when(F.array_contains(F.col(x),"yes"), F.lit("yes")).otherwise(F.lit("no"))
                                ).persist()
                        for phase in ["Phase4","approved"]:
                                
                                print("groupby for", phase)
                                #pre.groupBy(x).pivot(phase).count().select(x,"yes","no").sort(F.col(x).desc()).show()

                                array1 = np.delete(pre.groupBy(x).pivot(phase).count().select(x,"yes","no").sort(F.col(x).desc())
                                .fillna(0)
                                .toPandas()
                                .to_numpy(),
                                [0],
                                1,
                                )
                                total = np.sum(array1)
                                res_npPhaseX = np.array(array1, dtype=int)
                                resX = convertTuple(fisher_exact(res_npPhaseX, alternative="two-sided"))
                                resx_CI = convertTuple(
                                odds_ratio(res_npPhaseX).confidence_interval(confidence_level=0.95)
                                )
                                print(round(float(resX.split(",")[0]), 2),float(resX.split(",")[1]),
                                round(float(resx_CI.split(",")[0]), 2),
                                round(float(resx_CI.split(",")[1]), 2))
                                print("\n")
                                results.append([
                                        partitionByPValue,
                                        group_phase,
                                        group_row,
                                        phase,
                                        x,
                                        round(float(resX.split(",")[0]), 2), ## OR
                                        float(resX.split(",")[1]), ## pValue
                                        round(float(resx_CI.split(",")[0]), 2), ## Low CI
                                        round(float(resx_CI.split(",")[1]), 2), ## High CI
                                        total,
                                        array1])
                        pre.unpersist()
                else:
                        pre=new_benchmark.withColumn("approved_l", F.when(F.col("approvedDrug")==1, F.lit("yes")).otherwise(F.lit("no"))
                        ).filter(F.col("clinicalStudyId").isNotNull()
                        ).withColumn("agree_lowestPval", F.first("AgreeDrug").over(window_spec)
                        ).withColumn(
                        "clinicalPhase", ### no longer maxclinphase for T-D
                        F.max("clinicalPhase").over(Window.partitionBy(*group_phase_columns))
                        ).groupBy(*group_rows_columns).pivot("rightStudyType").agg(F.collect_set("agree_lowestPVal")
                        ).withColumn(
                                "Phase4",
                                F.when(F.col("clinicalPhase") == 4, F.lit("yes")).otherwise(F.lit("no"))
                        ).withColumn(
                                "Phase>=3",
                                F.when(F.col("clinicalPhase") >= 3, F.lit("yes")).otherwise(F.lit("no")),
                        ).withColumn(
                                "Phase>=2",
                                F.when(F.col("clinicalPhase") >= 2, F.lit("yes")).otherwise(F.lit("no")),
                        ).withColumn(
                                "Phase>=1",
                                F.when(F.col("clinicalPhase") >= 1, F.lit("yes")).otherwise(F.lit("no")),
                        ).withColumn(
                                x, 
                                F.when(F.array_contains(F.col(x),"yes"), F.lit("yes")).otherwise(F.lit("no"))
                        ).persist()
                        for phase in ["Phase4"]:
                                
                                print("groupby for", phase)
                                #pre.groupBy(x).pivot(phase).count().select(x,"yes","no").sort(F.col(x).desc()).show()

                                array1 = np.delete(pre.groupBy(x).pivot(phase).count().select(x,"yes","no").sort(F.col(x).desc())
                                .fillna(0)
                                .toPandas()
                                .to_numpy(),
                                [0],
                                1,
                                )
                                total = np.sum(array1)
                                res_npPhaseX = np.array(array1, dtype=int)
                                resX = convertTuple(fisher_exact(res_npPhaseX, alternative="two-sided"))
                                resx_CI = convertTuple(
                                odds_ratio(res_npPhaseX).confidence_interval(confidence_level=0.95)
                                )
                                print(round(float(resX.split(",")[0]), 2),float(resX.split(",")[1]),
                                round(float(resx_CI.split(",")[0]), 2),
                                round(float(resx_CI.split(",")[1]), 2))
                                print("\n")
                                results.append([
                                        partitionByPValue,
                                        group_phase,
                                        group_row,
                                        phase,
                                        x,
                                        round(float(resX.split(",")[0]), 2), ## OR
                                        float(resX.split(",")[1]), ## pValue
                                        round(float(resx_CI.split(",")[0]), 2), ## Low CI
                                        round(float(resx_CI.split(",")[1]), 2), ## High CI
                                        total,
                                        array1])
                        pre.unpersist()
df = pd.DataFrame(
results,
columns=["partitionByPValue",
        "partitionForPhase",
        "groupByForRows",
        "phase",
        "x",
        "OR",
        "pValue",
        "LowCI",
        "HighCI",
        "total",
        "array"

],
)

making group phases: ['targetId', 'diseaseId']
making group rows: ['targetId', 'diseaseId', 'clinicalPhase']
pqtl pqtl


groupby for Phase4


2.57 0.04795664956148016 0.99 7.05


making group rows: ['targetId', 'diseaseId', 'clinicalPhase', 'clinicalStudyId']
pqtl pqtl


groupby for Phase4


1.1 0.7410886427563347 0.7 1.81


making group rows: ['targetId', 'diseaseId', 'clinicalPhase', 'clinicalStudyId', 'drugId']
pqtl pqtl


groupby for Phase4


0.97 0.9099031384690608 0.63 1.58


making group rows: ['targetId', 'diseaseId', 'clinicalPhase', 'drugId']
pqtl pqtl


25/01/17 12:59:43 WARN CacheManager: Asked to cache already cached data.        


groupby for Phase4
2.33 0.008770489571684605 1.21 4.78


making group rows: ['targetId', 'diseaseId', 'clinicalPhase', 'approved_l']
pqtl pqtl


AnalysisException: Column 'approved' does not exist. Did you mean one of the following? [approved_l, Phase4, diseaseId, pqtl, targetId, Phase>=1, Phase>=2, Phase>=3, eqtl, sceqtl, scsqtl, sctuqtl, sqtl, tuqtl, clinicalPhase];
'Project [targetId#4743, diseaseId#4892, clinicalPhase#1520297, approved_l#1520196, eqtl#1521713, pqtl#1521715, sceqtl#1521717, scsqtl#1521719, sctuqtl#1521721, sqtl#1521723, tuqtl#1521725, Phase4#1521737, Phase>=3#1521750, Phase>=2#1521764, Phase>=1#1521779, CASE WHEN ('approved = 1) THEN yes ELSE no END AS approved#1521795]
+- Project [targetId#4743, diseaseId#4892, clinicalPhase#1520297, approved_l#1520196, eqtl#1521713, pqtl#1521715, sceqtl#1521717, scsqtl#1521719, sctuqtl#1521721, sqtl#1521723, tuqtl#1521725, Phase4#1521737, Phase>=3#1521750, Phase>=2#1521764, CASE WHEN (clinicalPhase#1520297 >= cast(1 as double)) THEN yes ELSE no END AS Phase>=1#1521779]
   +- Project [targetId#4743, diseaseId#4892, clinicalPhase#1520297, approved_l#1520196, eqtl#1521713, pq

In [10]:
df

,partitionByPValue,partitionForPhase,groupByForRows,phase,x,OR,pValue,LowCI,HighCI,total,array
0,"[targetId, diseaseId, rightStudyType]","[targetId, diseaseId]","[targetId, diseaseId, clinicalStudyId, clinica...",Phase4,pqtl,1.10,0.741089,0.70,1.81,4333,"[[290, 22], [3711, 310]]"
1,"[targetId, diseaseId, rightStudyType]","[targetId, diseaseId]","[targetId, diseaseId, clinicalStudyId, clinica...",Phase4,eqtl,1.38,0.006557,1.09,1.76,4333,"[[1718, 117], [2283, 215]]"
2,"[targetId, diseaseId, rightStudyType]","[targetId, diseaseId]","[targetId, diseaseId, clinicalStudyId, clinica...",Phase4,sqtl,1.39,0.138676,0.92,2.16,4333,"[[437, 27], [3564, 305]]"


In [22]:
import numpy as np
def convertTuple(tup):
    st = ",".join(map(str, tup))
    return st
from scipy.stats import fisher_exact
from scipy.stats.contingency import odds_ratio
from pyspark.sql.types import *

group1=["targetId", "diseaseId","maxClinPhase_TD"] ### canonical one
group2=["targetId", "diseaseId", "clinicalStudyId","clinicalPhase","maxClinPhase_TD","approvedDrug"]
group3=["targetId", "diseaseId", "maxClinPhase_TD","approvedDrug"] ## maxClinPhase_TD
group4=["targetId", "diseaseId","drugId", "maxClinPhase_TD","approvedDrug"] ### maxClinPhase_TD
group5=["targetId", "diseaseId","drugId","clinicalStudyId", "maxClinPhase_TD","approvedDrug"] ## maxClinPhase_TD
group6=["targetId", "diseaseId","clinicalPhase"]

# Dictionary to map group names to lists
group_mapping = {
    "group1": group1,
    "group2": group2,
    "group3": group3,
    "group4": group4,
    "group5": group5,
    "group6": group6
}
clinical_phases=["Phase4","approved"]
remove=["maxClinPhase_TD", "approvedDrug"]

value_analysis=["pqtl","eqtl","sqtl"]
# Prepare aggregation depending on the variable problem
window_spec = Window.partitionBy("targetId", "diseaseId", "projectId").orderBy(F.col("pValueExponent").asc())
for value in value_analysis:
    # Iterate over group mapping
    for group_name, group_columns in group_mapping.items():

        if "approvedDrug" in group_columns:

            print(f"Processing group: {group_name}", "with variables:", group_columns)
            filtered_group1 = [item for item in group_columns if item not in remove]
            print(filtered_group1,"for", value)
            x = value
            pre=new_benchmark.filter(F.col("clinicalStudyId").isNotNull()).withColumn(
                    "maxClinPhase_TD", ### no longer maxclinphase for T-D
                    F.max("clinicalPhase").over(Window.partitionBy(filtered_group1))).withColumn("agree_lowestPval", F.first("AgreeDrug").over(window_spec)
            ).groupBy(group_columns).pivot("rightStudyType").agg(F.collect_set("agree_lowestPVal")
            ).withColumn(
            "Phase4",
            F.when(F.col("maxClinPhase_TD") == 4, F.lit("yes")).otherwise(F.lit("no")),
            ).withColumn(
                "Phase>=3",
                F.when(F.col("maxClinPhase_TD") >= 3, F.lit("yes")).otherwise(F.lit("no")),
            ).withColumn(
                "Phase>=2",
                F.when(F.col("maxClinPhase_TD") >= 2, F.lit("yes")).otherwise(F.lit("no")),
            ).withColumn(
                "Phase>=1",
                F.when(F.col("maxClinPhase_TD") >= 1, F.lit("yes")).otherwise(F.lit("no")),
            ).withColumn(
            "approved",
                F.when(F.col("approvedDrug")==1, F.lit("yes")).otherwise(F.lit("no")),
            ).withColumn(
                x, 
                F.when(F.array_contains(F.col(x),"yes"), F.lit("yes")).otherwise(F.lit("no"))
            )

            for phase in clinical_phases:
                
                print("groupby for", phase)
                pre.groupBy(x).pivot(phase).count().select(x,"yes","no").sort(F.col(x).desc()).show()

                array1 = np.delete(pre.groupBy(x).pivot(phase).count().select(x,"yes","no").sort(F.col(x).desc())
                .fillna(0)
                .toPandas()
                .to_numpy(),
                [0],
                1,
                )
                total = np.sum(array1)
                res_npPhaseX = np.array(array1, dtype=int)
                resX = convertTuple(fisher_exact(res_npPhaseX, alternative="two-sided"))
                resx_CI = convertTuple(
                    odds_ratio(res_npPhaseX).confidence_interval(confidence_level=0.95)
                )
                print(round(float(resX.split(",")[0]), 2),float(resX.split(",")[1]),
                    round(float(resx_CI.split(",")[0]), 2),
                    round(float(resx_CI.split(",")[1]), 2))
                print("\n")
        elif "group6" == group_name:
            print(f"Processing group: {group_name}", "with variables:", group_columns)
            filtered_group1 = [item for item in group_columns if item not in remove]
            print(filtered_group1,"for", value)
            x = value
            pre=new_benchmark.filter(F.col("clinicalStudyId").isNotNull()).withColumn(
                    "maxClinPhase_TD", ### no longer maxclinphase for T-D
                    F.max("clinicalPhase").over(Window.partitionBy(filtered_group1))).withColumn("agree_lowestPval", F.first("AgreeDrug").over(window_spec)
            ).groupBy(group_columns).pivot("rightStudyType").agg(F.collect_set("agree_lowestPVal")
            ).withColumn(
            "Phase4",
            F.when(F.col("clinicalPhase") == 4, F.lit("yes")).otherwise(F.lit("no")),
            ).withColumn(
                "Phase>=3",
                F.when(F.col("clinicalPhase") >= 3, F.lit("yes")).otherwise(F.lit("no")),
            ).withColumn(
                "Phase>=2",
                F.when(F.col("clinicalPhase") >= 2, F.lit("yes")).otherwise(F.lit("no")),
            ).withColumn(
                "Phase>=1",
                F.when(F.col("clinicalPhase") >= 1, F.lit("yes")).otherwise(F.lit("no")),
            ).withColumn(
                x, 
                F.when(F.array_contains(F.col(x),"yes"), F.lit("yes")).otherwise(F.lit("no"))
            )
            
            for phase in ["phase4"]:
                print("groupby for just phase 4 because it does not have approvedDrug columns")
                pre.groupBy(x).pivot(phase).count().select(x,"yes","no").sort(F.col(x).desc()).show()

                array1 = np.delete(pre.groupBy(x).pivot(phase).count().select(x,"yes","no").sort(F.col(x).desc())
                .fillna(0)
                .toPandas()
                .to_numpy(),
                [0],
                1,
                )
                total = np.sum(array1)
                res_npPhaseX = np.array(array1, dtype=int)
                resX = convertTuple(fisher_exact(res_npPhaseX, alternative="two-sided"))
                resx_CI = convertTuple(
                    odds_ratio(res_npPhaseX).confidence_interval(confidence_level=0.95)
                )
                print(round(float(resX.split(",")[0]), 2),float(resX.split(",")[1]),
                    round(float(resx_CI.split(",")[0]), 2),
                    round(float(resx_CI.split(",")[1]), 2))
                print("\n")
        else: 
            print(f"Processing group: {group_name}", "with variables:", group_columns)
            filtered_group1 = [item for item in group_columns if item not in remove]
            print(filtered_group1,"for", value)
            x = value
            pre=new_benchmark.filter(F.col("clinicalStudyId").isNotNull()).withColumn(
                    "maxClinPhase_TD", ### no longer maxclinphase for T-D
                    F.max("clinicalPhase").over(Window.partitionBy(filtered_group1))).withColumn("agree_lowestPval", F.first("AgreeDrug").over(window_spec)
            ).groupBy(group_columns).pivot("rightStudyType").agg(F.collect_set("agree_lowestPVal")
            ).withColumn(
            "Phase4",
            F.when(F.col("maxClinPhase_TD") == 4, F.lit("yes")).otherwise(F.lit("no")),
            ).withColumn(
                "Phase>=3",
                F.when(F.col("maxClinPhase_TD") >= 3, F.lit("yes")).otherwise(F.lit("no")),
            ).withColumn(
                "Phase>=2",
                F.when(F.col("maxClinPhase_TD") >= 2, F.lit("yes")).otherwise(F.lit("no")),
            ).withColumn(
                "Phase>=1",
                F.when(F.col("maxClinPhase_TD") >= 1, F.lit("yes")).otherwise(F.lit("no")),
            ).withColumn(
                x, 
                F.when(F.array_contains(F.col(x),"yes"), F.lit("yes")).otherwise(F.lit("no"))
            )
            
            for phase in ["phase4"]:
                print("groupby for just phase 4 because it does not have approvedDrug columns")
                pre.groupBy(x).pivot(phase).count().select(x,"yes","no").sort(F.col(x).desc()).show()

                array1 = np.delete(pre.groupBy(x).pivot(phase).count().select(x,"yes","no").sort(F.col(x).desc())
                .fillna(0)
                .toPandas()
                .to_numpy(),
                [0],
                1,
                )
                total = np.sum(array1)
                res_npPhaseX = np.array(array1, dtype=int)
                resX = convertTuple(fisher_exact(res_npPhaseX, alternative="two-sided"))
                resx_CI = convertTuple(
                    odds_ratio(res_npPhaseX).confidence_interval(confidence_level=0.95)
                )
                print(round(float(resX.split(",")[0]), 2),float(resX.split(",")[1]),
                    round(float(resx_CI.split(",")[0]), 2),
                    round(float(resx_CI.split(",")[1]), 2))
                print("\n")

Processing group: group1 with variables: ['targetId', 'diseaseId', 'maxClinPhase_TD']
['targetId', 'diseaseId'] for pqtl


groupby for just phase 4 because it does not have approvedDrug columns


+----+---+---+
|pqtl|yes| no|
+----+---+---+
| yes| 16|  9|
|  no| 58| 84|
+----+---+---+



2.57 0.04795664956148016 0.99 7.05


Processing group: group2 with variables: ['targetId', 'diseaseId', 'clinicalStudyId', 'clinicalPhase', 'maxClinPhase_TD', 'approvedDrug']
['targetId', 'diseaseId', 'clinicalStudyId', 'clinicalPhase'] for pqtl


groupby for Phase4


...

+----+----+----+
|sqtl| yes|  no|
+----+----+----+
| yes| 953| 238|
|  no|2731|1238|
+----+----+----+



1.82 1.761092533228378e-14 1.55 2.13


Processing group: group6 with variables: ['targetId', 'diseaseId', 'clinicalPhase']
['targetId', 'diseaseId', 'clinicalPhase'] for sqtl


groupby for just phase 4 because it does not have approvedDrug columns


+----+---+---+
|sqtl|yes| no|
+----+---+---+
| yes| 13| 66|
|  no| 61|234|
+----+---+---+



0.76 0.5246665763693217 0.36 1.5




##### See how the benchmark varies regarding maxclinphase made in chembl_indication

In [ ]:
#### build list of comparison and prediction columns
rows=comparisons_df_iterative(disdic,variables_study[2])[0]
#### prepare aggregation depending on the variable problem
window_spec = Window.partitionBy("targetId","diseaseId","rightStudyType").orderBy(F.col("pValueExponent").asc())
#### take directionality from lowest p value
bench2=benchmark.withColumn("agree_lowestPval", F.first("AgreeDrug").over(window_spec)
    ).groupBy("targetId", "diseaseId", "clinicalStudyId","clinicalPhase","maxClinPhase","approvedDrug").pivot("rightStudyType").agg(F.collect_set("agree_lowestPVal")
    ).withColumn(
    "Phase4",
    F.when(F.col("maxClinPhase") == 4, F.lit("yes")).otherwise(F.lit("no")),
    ).withColumn(
        "Phase>=3",
        F.when(F.col("maxClinPhase") >= 3, F.lit("yes")).otherwise(F.lit("no")),
    ).withColumn(
        "Phase>=2",
        F.when(F.col("maxClinPhase") >= 2, F.lit("yes")).otherwise(F.lit("no")),
    ).withColumn(
        "Phase>=1",
        F.when(F.col("maxClinPhase") >= 1, F.lit("yes")).otherwise(F.lit("no")),
    ).withColumn(
    "approved",
        F.when(F.col("approvedDrug")==1, F.lit("yes")).otherwise(F.lit("no")),
    )
#### build columns yes/no for each distinct value in the column variable

bench3=benchmark.withColumn("agree_lowestPval", F.first("AgreeDrug").over(window_spec)
    ).withColumn(
            "maxClinPhase_TD", ### maxclinphase for T-D
            F.max("clinicalPhase").over(Window.partitionBy("targetId", "diseaseId")),
    ).groupBy("targetId", "diseaseId", "maxClinPhase_TD","approvedDrug").pivot("rightStudyType").agg(F.collect_set("agree_lowestPVal")
    ).withColumn(
    "Phase4",
    F.when(F.col("maxClinPhase_TD") == 4, F.lit("yes")).otherwise(F.lit("no")),
    ).withColumn(
        "Phase>=3",
        F.when(F.col("maxClinPhase_TD") >= 3, F.lit("yes")).otherwise(F.lit("no")),
    ).withColumn(
        "Phase>=2",
        F.when(F.col("maxClinPhase_TD") >= 2, F.lit("yes")).otherwise(F.lit("no")),
    ).withColumn(
        "Phase>=1",
        F.when(F.col("maxClinPhase_TD") >= 1, F.lit("yes")).otherwise(F.lit("no")),
    ).withColumn(
    "approved",
        F.when(F.col("approvedDrug")==1, F.lit("yes")).otherwise(F.lit("no")),
    )
#### build columns yes/no for each distinct value in the column variable
bench4=benchmark.withColumn("agree_lowestPval", F.first("AgreeDrug").over(window_spec)
    ).withColumn(
            "maxClinPhase_TD", ### no longer maxclinphase for T-D
            F.max("clinicalPhase").over(Window.partitionBy("targetId", "diseaseId")),
    ).groupBy("targetId", "diseaseId","drugId", "maxClinPhase_TD","approvedDrug").pivot("rightStudyType").agg(F.collect_set("agree_lowestPVal")
    ).withColumn(
    "Phase4",
    F.when(F.col("maxClinPhase_TD") == 4, F.lit("yes")).otherwise(F.lit("no")),
    ).withColumn(
        "Phase>=3",
        F.when(F.col("maxClinPhase_TD") >= 3, F.lit("yes")).otherwise(F.lit("no")),
    ).withColumn(
        "Phase>=2",
        F.when(F.col("maxClinPhase_TD") >= 2, F.lit("yes")).otherwise(F.lit("no")),
    ).withColumn(
        "Phase>=1",
        F.when(F.col("maxClinPhase_TD") >= 1, F.lit("yes")).otherwise(F.lit("no")),
    ).withColumn(
    "approved",
        F.when(F.col("approvedDrug")==1, F.lit("yes")).otherwise(F.lit("no")),
    )

In [ ]:
x="pqtl"
print("")
bench2.withColumn(
        x, 
        F.when(F.array_contains(F.col(x),"yes"), F.lit("yes")).otherwise(F.lit("no"))
    ).groupBy(x).pivot("phase4").count().select(x,"yes","no").sort(F.col(x).desc()).show()
bench3.withColumn(
        x, 
        F.when(F.array_contains(F.col(x),"yes"), F.lit("yes")).otherwise(F.lit("no"))
    ).groupBy(x).pivot("phase4").count().select(x,"yes","no").sort(F.col(x).desc()).show()

bench4.withColumn(
        x, 
        F.when(F.array_contains(F.col(x),"yes"), F.lit("yes")).otherwise(F.lit("no"))
    ).groupBy(x).pivot("phase4").count().select(x,"yes","no").sort(F.col(x).desc()).show()

bench4.withColumn(
        x, 
        F.when(F.array_contains(F.col(x),"yes"), F.lit("yes")).otherwise(F.lit("no"))
    ).groupBy(x).pivot("approved").count().select(x,"yes","no").sort(F.col(x).desc()).show()

+----+----+----+
|pqtl| yes|  no|
+----+----+----+
| yes|  63| 265|
|  no|1142|2944|
+----+----+----+



+----+---+---+
|pqtl|yes| no|
+----+---+---+
| yes| 21|  6|
|  no| 78| 77|
+----+---+---+



+----+---+---+
|pqtl|yes| no|
+----+---+---+
| yes| 75|  7|
|  no|444|152|
+----+---+---+



+----+---+---+
|pqtl|yes| no|
+----+---+---+
| yes| 64| 18|
|  no|384|212|
+----+---+---+



In [ ]:
x="eqtl"
bench2.withColumn(
        x, 
        F.when(F.array_contains(F.col(x),"yes"), F.lit("yes")).otherwise(F.lit("no"))
    ).groupBy(x).pivot("phase4").count().show()
bench3.withColumn(
        x, 
        F.when(F.array_contains(F.col(x),"yes"), F.lit("yes")).otherwise(F.lit("no"))
    ).groupBy(x).pivot("phase4").count().show()

bench4.withColumn(
        x, 
        F.when(F.array_contains(F.col(x),"yes"), F.lit("yes")).otherwise(F.lit("no"))
    ).groupBy(x).pivot("phase4").count().show()


+----+----+---+
|eqtl|  no|yes|
+----+----+---+
|  no|2121|886|
| yes|1088|319|
+----+----+---+



+----+---+---+
|eqtl| no|yes|
+----+---+---+
|  no| 49| 63|
| yes| 34| 36|
+----+---+---+



+----+---+---+
|eqtl| no|yes|
+----+---+---+
|  no| 95|280|
| yes| 64|239|
+----+---+---+



In [38]:
x="sqtl"
bench2.withColumn(
        x, 
        F.when(F.array_contains(F.col(x),"yes"), F.lit("yes")).otherwise(F.lit("no"))
    ).groupBy(x).pivot("phase4").count().show()
bench3.withColumn(
        x, 
        F.when(F.array_contains(F.col(x),"yes"), F.lit("yes")).otherwise(F.lit("no"))
    ).groupBy(x).pivot("phase4").count().show()

bench4.withColumn(
        x, 
        F.when(F.array_contains(F.col(x),"yes"), F.lit("yes")).otherwise(F.lit("no"))
    ).groupBy(x).pivot("phase4").count().show()

+----+----+---+
|sqtl|  no|yes|
+----+----+---+
|  no|2688|969|
| yes| 521|236|
+----+----+---+



+----+---+---+
|sqtl| no|yes|
+----+---+---+
|  no| 74| 88|
| yes|  9| 11|
+----+---+---+



+----+---+---+
|sqtl| no|yes|
+----+---+---+
|  no|147|454|
| yes| 12| 65|
+----+---+---+



### now with studies that we know they perform well

In [30]:
disdic

{'ROSMAP': 'projectId',
 'BrainSeq': 'projectId',
 'BLUEPRINT': 'projectId',
 'GEUVADIS': 'projectId',
 'TwinsUK': 'projectId',
 'GTEx': 'projectId',
 'FUSION': 'projectId',
 'Lepik_2017': 'projectId',
 'Fairfax_2014': 'projectId',
 'CEDAR': 'projectId',
 'OneK1K': 'projectId',
 'UKB_PPP_EUR': 'projectId',
 'Schmiedel_2018': 'projectId',
 'Perez_2022': 'projectId',
 'Quach_2016': 'projectId',
 'Alasoo_2018': 'projectId',
 'Gilchrist_2021': 'projectId',
 'CommonMind': 'projectId',
 'Sun_2018': 'projectId',
 'Braineac2': 'projectId',
 'Peng_2018': 'projectId',
 'Bossini-Castillo_2019': 'projectId',
 'Nedelec_2016': 'projectId',
 'Steinberg_2020': 'projectId',
 'Jerber_2021': 'projectId',
 'HipSci': 'projectId',
 'GENCORD': 'projectId',
 'Kim-Hellmuth_2017': 'projectId',
 'CAP': 'projectId',
 'iPSCORE': 'projectId',
 'PhLiPS': 'projectId',
 'Walker_2019': 'projectId',
 'Aygun_2021': 'projectId',
 'PISA': 'projectId',
 'Cytoimmgen': 'projectId',
 'van_de_Bunt_2015': 'projectId',
 'Nathan_2

In [31]:
#### build list of comparison and prediction columns
rows=comparisons_df_iterative(disdic,variables_study[2])[0]
#### prepare aggregation depending on the variable problem
window_spec = Window.partitionBy("targetId","diseaseId","projectId").orderBy(F.col("pValueExponent").asc())
#### take directionality from lowest p value
bench2=benchmark.withColumn("agree_lowestPval", F.first("AgreeDrug").over(window_spec)
    ).groupBy("targetId", "diseaseId", "clinicalStudyId","clinicalPhase","maxClinPhase","approvedDrug").pivot("projectId").agg(F.collect_set("agree_lowestPVal")
    ).withColumn(
    "Phase4",
    F.when(F.col("maxClinPhase") == 4, F.lit("yes")).otherwise(F.lit("no")),
    ).withColumn(
        "Phase>=3",
        F.when(F.col("maxClinPhase") >= 3, F.lit("yes")).otherwise(F.lit("no")),
    ).withColumn(
        "Phase>=2",
        F.when(F.col("maxClinPhase") >= 2, F.lit("yes")).otherwise(F.lit("no")),
    ).withColumn(
        "Phase>=1",
        F.when(F.col("maxClinPhase") >= 1, F.lit("yes")).otherwise(F.lit("no")),
    ).withColumn(
    "approved",
        F.when(F.col("approvedDrug")==1, F.lit("yes")).otherwise(F.lit("no")),
    )
#### build columns yes/no for each distinct value in the column variable

bench3=benchmark.withColumn("agree_lowestPval", F.first("AgreeDrug").over(window_spec)
    ).withColumn(
            "maxClinPhase_TD", ### no longer maxclinphase for T-D
            F.max("clinicalPhase").over(Window.partitionBy("targetId", "diseaseId")),
    ).groupBy("targetId", "diseaseId", "maxClinPhase_TD","approvedDrug").pivot("projectId").agg(F.collect_set("agree_lowestPVal")
    ).withColumn(
    "Phase4",
    F.when(F.col("maxClinPhase_TD") == 4, F.lit("yes")).otherwise(F.lit("no")),
    ).withColumn(
        "Phase>=3",
        F.when(F.col("maxClinPhase_TD") >= 3, F.lit("yes")).otherwise(F.lit("no")),
    ).withColumn(
        "Phase>=2",
        F.when(F.col("maxClinPhase_TD") >= 2, F.lit("yes")).otherwise(F.lit("no")),
    ).withColumn(
        "Phase>=1",
        F.when(F.col("maxClinPhase_TD") >= 1, F.lit("yes")).otherwise(F.lit("no")),
    ).withColumn(
    "approved",
        F.when(F.col("approvedDrug")==1, F.lit("yes")).otherwise(F.lit("no")),
    )
#### build columns yes/no for each distinct value in the column variable
bench4=benchmark.withColumn("agree_lowestPval", F.first("AgreeDrug").over(window_spec)
    ).withColumn(
            "maxClinPhase_TD", ### no longer maxclinphase for T-D
            F.max("clinicalPhase").over(Window.partitionBy("targetId", "diseaseId")),
    ).groupBy("targetId", "diseaseId","drugId", "maxClinPhase_TD","approvedDrug").pivot("projectId").agg(F.collect_set("agree_lowestPVal")
    ).withColumn(
    "Phase4",
    F.when(F.col("maxClinPhase_TD") == 4, F.lit("yes")).otherwise(F.lit("no")),
    ).withColumn(
        "Phase>=3",
        F.when(F.col("maxClinPhase_TD") >= 3, F.lit("yes")).otherwise(F.lit("no")),
    ).withColumn(
        "Phase>=2",
        F.when(F.col("maxClinPhase_TD") >= 2, F.lit("yes")).otherwise(F.lit("no")),
    ).withColumn(
        "Phase>=1",
        F.when(F.col("maxClinPhase_TD") >= 1, F.lit("yes")).otherwise(F.lit("no")),
    ).withColumn(
    "approved",
        F.when(F.col("approvedDrug")==1, F.lit("yes")).otherwise(F.lit("no")),
    )

In [33]:
x="UKB_PPP_EUR"
bench2.withColumn(
        x, 
        F.when(F.array_contains(F.col(x),"yes"), F.lit("yes")).otherwise(F.lit("no"))
    ).groupBy(x).pivot("phase4").count().show()
bench3.withColumn(
        x, 
        F.when(F.array_contains(F.col(x),"yes"), F.lit("yes")).otherwise(F.lit("no"))
    ).groupBy(x).pivot("phase4").count().show()

bench4.withColumn(
        x, 
        F.when(F.array_contains(F.col(x),"yes"), F.lit("yes")).otherwise(F.lit("no"))
    ).groupBy(x).pivot("phase4").count().show()

+-----------+----+----+
|UKB_PPP_EUR|  no| yes|
+-----------+----+----+
|         no|2944|1142|
|        yes| 265|  63|
+-----------+----+----+



+-----------+---+---+
|UKB_PPP_EUR| no|yes|
+-----------+---+---+
|         no| 77| 78|
|        yes|  6| 21|
+-----------+---+---+



+-----------+---+---+
|UKB_PPP_EUR| no|yes|
+-----------+---+---+
|         no|152|444|
|        yes|  7| 75|
+-----------+---+---+



### build report to analyse different comparisons

In [ ]:
### including clinicalStudyId to explode the data

#### build list of comparison and prediction columns
rows=comparisons_df_iterative(disdic,variables_study[2])[0]
#### prepare aggregation depending on the variable problem
window_spec = Window.partitionBy("targetId","diseaseId","projectId").orderBy(F.col("pValueExponent").asc())
#### take directionality from lowest p value
bench2=benchmark.withColumn("agree_lowestPval", F.first("AgreeDrug").over(window_spec)
    ).groupBy("targetId", "diseaseId", "clinicalStudyId","clinicalPhase","maxClinPhase","approvedDrug").pivot("projectId").agg(F.collect_set("agree_lowestPVal")
    ).withColumn(
    "Phase4",
    F.when(F.col("maxClinPhase") == 4, F.lit("yes")).otherwise(F.lit("no")),
    ).withColumn(
        "Phase>=3",
        F.when(F.col("maxClinPhase") >= 3, F.lit("yes")).otherwise(F.lit("no")),
    ).withColumn(
        "Phase>=2",
        F.when(F.col("maxClinPhase") >= 2, F.lit("yes")).otherwise(F.lit("no")),
    ).withColumn(
        "Phase>=1",
        F.when(F.col("maxClinPhase") >= 1, F.lit("yes")).otherwise(F.lit("no")),
    ).withColumn(
    "approved",
        F.when(F.col("approvedDrug")==1, F.lit("yes")).otherwise(F.lit("no")),
    )
#### build columns yes/no for each distinct value in the column variable

bench3=benchmark.withColumn("agree_lowestPval", F.first("AgreeDrug").over(window_spec)
    ).withColumn(
            "maxClinPhase_TD", ### no longer maxclinphase for T-D
            F.max("clinicalPhase").over(Window.partitionBy("targetId", "diseaseId")),
    ).groupBy("targetId", "diseaseId", "maxClinPhase_TD","approvedDrug").pivot("projectId").agg(F.collect_set("agree_lowestPVal")
    ).withColumn(
    "Phase4",
    F.when(F.col("maxClinPhase_TD") == 4, F.lit("yes")).otherwise(F.lit("no")),
    ).withColumn(
        "Phase>=3",
        F.when(F.col("maxClinPhase_TD") >= 3, F.lit("yes")).otherwise(F.lit("no")),
    ).withColumn(
        "Phase>=2",
        F.when(F.col("maxClinPhase_TD") >= 2, F.lit("yes")).otherwise(F.lit("no")),
    ).withColumn(
        "Phase>=1",
        F.when(F.col("maxClinPhase_TD") >= 1, F.lit("yes")).otherwise(F.lit("no")),
    ).withColumn(
    "approved",
        F.when(F.col("approvedDrug")==1, F.lit("yes")).otherwise(F.lit("no")),
    )
#### build columns yes/no for each distinct value in the column variable
bench4=benchmark.withColumn("agree_lowestPval", F.first("AgreeDrug").over(window_spec)
    ).withColumn(
            "maxClinPhase_TD", ### no longer maxclinphase for T-D
            F.max("clinicalPhase").over(Window.partitionBy("targetId", "diseaseId")),
    ).groupBy("targetId", "diseaseId","drugId", "maxClinPhase_TD","approvedDrug").pivot("projectId").agg(F.collect_set("agree_lowestPVal")
    ).withColumn(
    "Phase4",
    F.when(F.col("maxClinPhase_TD") == 4, F.lit("yes")).otherwise(F.lit("no")),
    ).withColumn(
        "Phase>=3",
        F.when(F.col("maxClinPhase_TD") >= 3, F.lit("yes")).otherwise(F.lit("no")),
    ).withColumn(
        "Phase>=2",
        F.when(F.col("maxClinPhase_TD") >= 2, F.lit("yes")).otherwise(F.lit("no")),
    ).withColumn(
        "Phase>=1",
        F.when(F.col("maxClinPhase_TD") >= 1, F.lit("yes")).otherwise(F.lit("no")),
    ).withColumn(
    "approved",
        F.when(F.col("approvedDrug")==1, F.lit("yes")).otherwise(F.lit("no")),
    )
bench5=benchmark.withColumn("agree_lowestPval", F.first("AgreeDrug").over(window_spec)
    ).withColumn(
            "maxClinPhase_TD", ### no longer maxclinphase for T-D
            F.max("clinicalPhase").over(Window.partitionBy("targetId", "diseaseId")),
    ).groupBy("targetId", "diseaseId","drugId","clinicalStudyId", "maxClinPhase_TD","approvedDrug").pivot("projectId").agg(F.collect_set("agree_lowestPVal")
    ).withColumn(
    "Phase4",
    F.when(F.col("maxClinPhase_TD") == 4, F.lit("yes")).otherwise(F.lit("no")),
    ).withColumn(
        "Phase>=3",
        F.when(F.col("maxClinPhase_TD") >= 3, F.lit("yes")).otherwise(F.lit("no")),
    ).withColumn(
        "Phase>=2",
        F.when(F.col("maxClinPhase_TD") >= 2, F.lit("yes")).otherwise(F.lit("no")),
    ).withColumn(
        "Phase>=1",
        F.when(F.col("maxClinPhase_TD") >= 1, F.lit("yes")).otherwise(F.lit("no")),
    ).withColumn(
    "approved",
        F.when(F.col("approvedDrug")==1, F.lit("yes")).otherwise(F.lit("no")),
    )

In [ ]:
## al clinical phases with drugId
x="UKB_PPP_EUR"
bench2.withColumn(
        x, 
        F.when(F.array_contains(F.col(x),"yes"), F.lit("yes")).otherwise(F.lit("no"))
    ).groupBy(x).pivot("phase4").count().select(x,"yes","no").sort(F.col(x).desc()).show()
bench3.withColumn(
        x, 
        F.when(F.array_contains(F.col(x),"yes"), F.lit("yes")).otherwise(F.lit("no"))
    ).groupBy(x).pivot("phase4").count().select(x,"yes","no").sort(F.col(x).desc()).show()

bench4.withColumn(
        x, 
        F.when(F.array_contains(F.col(x),"yes"), F.lit("yes")).otherwise(F.lit("no"))
    ).groupBy(x).pivot("phase4").count().select(x,"yes","no").sort(F.col(x).desc()).show()

bench5.withColumn(
        x, 
        F.when(F.array_contains(F.col(x),"yes"), F.lit("yes")).otherwise(F.lit("no"))
    ).groupBy(x).pivot("phase4").count().select(x,"yes","no").sort(F.col(x).desc()).show()

+-----------+----+----+
|UKB_PPP_EUR|  no| yes|
+-----------+----+----+
|         no|2944|1142|
|        yes| 265|  63|
+-----------+----+----+



+-----------+---+---+
|UKB_PPP_EUR| no|yes|
+-----------+---+---+
|         no|266| 78|
|        yes| 39| 21|
+-----------+---+---+



+-----------+---+---+
|UKB_PPP_EUR| no|yes|
+-----------+---+---+
|         no|723|323|
|        yes| 94| 47|
+-----------+---+---+



+-----------+----+----+
|UKB_PPP_EUR|  no| yes|
+-----------+----+----+
|         no|3471|1630|
|        yes| 270|  86|
+-----------+----+----+



In [43]:
### using maximum clinical phases per target-disease-drugId 


#### build list of comparison and prediction columns
rows=comparisons_df_iterative(disdic,variables_study[2])[0]
#### prepare aggregation depending on the variable problem
window_spec = Window.partitionBy("targetId","diseaseId","projectId").orderBy(F.col("pValueExponent").asc())
#### take directionality from lowest p value
bench2=benchmark.withColumn("agree_lowestPval", F.first("AgreeDrug").over(window_spec)
    ).groupBy("targetId", "diseaseId", "clinicalStudyId","clinicalPhase","maxClinPhase","approvedDrug").pivot("projectId").agg(F.collect_set("agree_lowestPVal")
    ).withColumn(
    "Phase4",
    F.when(F.col("maxClinPhase") == 4, F.lit("yes")).otherwise(F.lit("no")),
    ).withColumn(
        "Phase>=3",
        F.when(F.col("maxClinPhase") >= 3, F.lit("yes")).otherwise(F.lit("no")),
    ).withColumn(
        "Phase>=2",
        F.when(F.col("maxClinPhase") >= 2, F.lit("yes")).otherwise(F.lit("no")),
    ).withColumn(
        "Phase>=1",
        F.when(F.col("maxClinPhase") >= 1, F.lit("yes")).otherwise(F.lit("no")),
    ).withColumn(
    "approved",
        F.when(F.col("approvedDrug")==1, F.lit("yes")).otherwise(F.lit("no")),
    )
#### build columns yes/no for each distinct value in the column variable

bench3=benchmark.withColumn("agree_lowestPval", F.first("AgreeDrug").over(window_spec)
    ).withColumn(
            "maxClinPhase_TD", ### no longer maxclinphase for T-D
            F.max("clinicalPhase").over(Window.partitionBy("targetId", "diseaseId")),
    ).groupBy("targetId", "diseaseId", "maxClinPhase","approvedDrug").pivot("projectId").agg(F.collect_set("agree_lowestPVal")
    ).withColumn(
    "Phase4",
    F.when(F.col("maxClinPhase") == 4, F.lit("yes")).otherwise(F.lit("no")),
    ).withColumn(
        "Phase>=3",
        F.when(F.col("maxClinPhase") >= 3, F.lit("yes")).otherwise(F.lit("no")),
    ).withColumn(
        "Phase>=2",
        F.when(F.col("maxClinPhase") >= 2, F.lit("yes")).otherwise(F.lit("no")),
    ).withColumn(
        "Phase>=1",
        F.when(F.col("maxClinPhase") >= 1, F.lit("yes")).otherwise(F.lit("no")),
    ).withColumn(
    "approved",
        F.when(F.col("approvedDrug")==1, F.lit("yes")).otherwise(F.lit("no")),
    )
#### build columns yes/no for each distinct value in the column variable
bench4=benchmark.withColumn("agree_lowestPval", F.first("AgreeDrug").over(window_spec)
    ).withColumn(
            "maxClinPhase_TD", ### no longer maxclinphase for T-D
            F.max("clinicalPhase").over(Window.partitionBy("targetId", "diseaseId","drugId")),
    ).groupBy("targetId", "diseaseId","drugId", "maxClinPhase_TD","approvedDrug").pivot("projectId").agg(F.collect_set("agree_lowestPVal")
    ).withColumn(
    "Phase4",
    F.when(F.col("maxClinPhase_TD") == 4, F.lit("yes")).otherwise(F.lit("no")),
    ).withColumn(
        "Phase>=3",
        F.when(F.col("maxClinPhase_TD") >= 3, F.lit("yes")).otherwise(F.lit("no")),
    ).withColumn(
        "Phase>=2",
        F.when(F.col("maxClinPhase_TD") >= 2, F.lit("yes")).otherwise(F.lit("no")),
    ).withColumn(
        "Phase>=1",
        F.when(F.col("maxClinPhase_TD") >= 1, F.lit("yes")).otherwise(F.lit("no")),
    ).withColumn(
    "approved",
        F.when(F.col("approvedDrug")==1, F.lit("yes")).otherwise(F.lit("no")),
    )
bench5=benchmark.withColumn("agree_lowestPval", F.first("AgreeDrug").over(window_spec)
    ).withColumn(
            "maxClinPhase_TD", ### no longer maxclinphase for T-D
            F.max("clinicalPhase").over(Window.partitionBy("targetId", "diseaseId","drugId")),
    ).groupBy("targetId", "diseaseId","drugId","clinicalStudyId", "maxClinPhase_TD","approvedDrug").pivot("projectId").agg(F.collect_set("agree_lowestPVal")
    ).withColumn(
    "Phase4",
    F.when(F.col("maxClinPhase_TD") == 4, F.lit("yes")).otherwise(F.lit("no")),
    ).withColumn(
        "Phase>=3",
        F.when(F.col("maxClinPhase_TD") >= 3, F.lit("yes")).otherwise(F.lit("no")),
    ).withColumn(
        "Phase>=2",
        F.when(F.col("maxClinPhase_TD") >= 2, F.lit("yes")).otherwise(F.lit("no")),
    ).withColumn(
        "Phase>=1",
        F.when(F.col("maxClinPhase_TD") >= 1, F.lit("yes")).otherwise(F.lit("no")),
    ).withColumn(
    "approved",
        F.when(F.col("approvedDrug")==1, F.lit("yes")).otherwise(F.lit("no")),
    )

In [44]:
x="UKB_PPP_EUR"
bench2.withColumn(
        x, 
        F.when(F.array_contains(F.col(x),"yes"), F.lit("yes")).otherwise(F.lit("no"))
    ).groupBy(x).pivot("phase4").count().show()
bench3.withColumn(
        x, 
        F.when(F.array_contains(F.col(x),"yes"), F.lit("yes")).otherwise(F.lit("no"))
    ).groupBy(x).pivot("phase4").count().show()

bench4.withColumn(
        x, 
        F.when(F.array_contains(F.col(x),"yes"), F.lit("yes")).otherwise(F.lit("no"))
    ).groupBy(x).pivot("phase4").count().show()

bench5.withColumn(
        x, 
        F.when(F.array_contains(F.col(x),"yes"), F.lit("yes")).otherwise(F.lit("no"))
    ).groupBy(x).pivot("phase4").count().show()

+-----------+----+----+
|UKB_PPP_EUR|  no| yes|
+-----------+----+----+
|         no|2944|1142|
|        yes| 265|  63|
+-----------+----+----+



+-----------+---+---+
|UKB_PPP_EUR| no|yes|
+-----------+---+---+
|         no|266| 78|
|        yes| 39| 21|
+-----------+---+---+



+-----------+---+---+
|UKB_PPP_EUR| no|yes|
+-----------+---+---+
|         no|273|323|
|        yes| 35| 47|
+-----------+---+---+



+-----------+---+----+
|UKB_PPP_EUR| no| yes|
+-----------+---+----+
|         no|658|4443|
|        yes| 76| 280|
+-----------+---+----+



In [35]:
x="UKB_PPP_EUR"
bench2.withColumn(
        x, 
        F.when(F.array_contains(F.col(x),"yes"), F.lit("yes")).otherwise(F.lit("no"))
    ).groupBy(x).pivot("phase4").count().show()
bench3.withColumn(
        x, 
        F.when(F.array_contains(F.col(x),"yes"), F.lit("yes")).otherwise(F.lit("no"))
    ).groupBy(x).pivot("phase4").count().show()

bench4.withColumn(
        x, 
        F.when(F.array_contains(F.col(x),"yes"), F.lit("yes")).otherwise(F.lit("no"))
    ).groupBy(x).pivot("phase4").count().show()

bench5.withColumn(
        x, 
        F.when(F.array_contains(F.col(x),"yes"), F.lit("yes")).otherwise(F.lit("no"))
    ).groupBy(x).pivot("phase4").count().show()

+-----------+----+----+
|UKB_PPP_EUR|  no| yes|
+-----------+----+----+
|         no|2944|1142|
|        yes| 265|  63|
+-----------+----+----+



+-----------+---+---+
|UKB_PPP_EUR| no|yes|
+-----------+---+---+
|         no| 77| 78|
|        yes|  6| 21|
+-----------+---+---+



+-----------+---+---+
|UKB_PPP_EUR| no|yes|
+-----------+---+---+
|         no|152|444|
|        yes|  7| 75|
+-----------+---+---+



+-----------+---+----+
|UKB_PPP_EUR| no| yes|
+-----------+---+----+
|         no|276|4825|
|        yes|  9| 347|
+-----------+---+----+



In [ ]:
####2 Define agregation function
import pandas as pd
import numpy as np
from scipy.stats import fisher_exact
from scipy.stats.contingency import odds_ratio
from pyspark.sql.types import *

def convertTuple(tup):
    st = ",".join(map(str, tup))
    return st

#####3 run in a function
def aggregations_original(
    df,
    data,
    listado,
    comparisonColumn,
    comparisonType,
    predictionColumn,
    predictionType,
    today_date,
):
    wComparison = Window.partitionBy(comparisonColumn)
    wPrediction = Window.partitionBy(predictionColumn)
    wPredictionComparison = Window.partitionBy(comparisonColumn, predictionColumn)
    results = []
    # uniqIds = df.select("targetId", "diseaseId").distinct().count()
    out = (
        df.withColumn("comparisonType", F.lit(comparisonType))
        .withColumn("dataset", F.lit(data))
        .withColumn("predictionType", F.lit(predictionType))
        # .withColumn("total", F.lit(uniqIds))
        .withColumn("a", F.count("targetId").over(wPredictionComparison))
        .withColumn("comparisonColumn", F.lit(comparisonColumn))
        .withColumn("predictionColumnValue", F.lit(predictionColumn))
        .withColumn(
            "predictionTotal",
            F.count("targetId").over(wPrediction),
        )
        .withColumn(
            "comparisonTotal",
            F.count("targetId").over(wComparison),
        )
        .select(
            F.col(predictionColumn).alias("prediction"),
            F.col(comparisonColumn).alias("comparison"),
            "dataset",
            "comparisonColumn",
            "predictionColumnValue",
            "comparisonType",
            "predictionType",
            "a",
            "predictionTotal",
            "comparisonTotal",
        )
        .filter(F.col("prediction").isNotNull())
        .filter(F.col("comparison").isNotNull())
        .distinct()
    )
    '''
    out.write.mode("overwrite").parquet(
        "gs://ot-team/jroldan/"
        + str(
            today_date
            + "_"
            + "analysis/"
            + data
            # + "_propagated"
            + "/"
            + comparisonColumn
            + "_"
            + comparisonType
            + "_"
            + predictionColumn
            + ".parquet"
        )
    )
    '''
    
    listado.append(
        "gs://ot-team/jroldan/"
        + str(
            today_date
            + "_"
            + "analysis/"
            + data
            # + "_propagated"
            + "/"
            + comparisonColumn
            + "_"
            + comparisonType
            + "_"
            + predictionColumn
            + ".parquet"
        )
    )
    path = "gs://ot-team/jroldan/" + str(
        today_date
        + "_"
        + "analysis/"
        + data
        # + "_propagated"
        + "/"
        + comparisonColumn
        + "_"
        + comparisonType
        + "_"
        + predictionColumn
        + ".parquet"
    )
    print(path)
    ### making analysis
    array1 = np.delete(
        out.join(full_data, on=["prediction", "comparison"], how="outer")
        .groupBy("comparison")
        .pivot("prediction")
        .agg(F.first("a"))
        .sort(F.col("comparison").desc())
        .select("comparison", "yes", "no")
        .fillna(0)
        .toPandas()
        .to_numpy(),
        [0],
        1,
    )
    total = np.sum(array1)
    res_npPhaseX = np.array(array1, dtype=int)
    resX = convertTuple(fisher_exact(res_npPhaseX, alternative="two-sided"))
    resx_CI = convertTuple(
        odds_ratio(res_npPhaseX).confidence_interval(confidence_level=0.95)
    )

    result_st.append(resX)
    result_ci.append(resx_CI)
    (rs_result, rs_ci) = relative_success(array1)
    results.extend(
        [
            comparisonType,
            comparisonColumn,
            predictionColumn,
            round(float(resX.split(",")[0]), 2),
            float(resX.split(",")[1]),
            round(float(resx_CI.split(",")[0]), 2),
            round(float(resx_CI.split(",")[1]), 2),
            str(total),
            np.array(res_npPhaseX).tolist(),
            round(float(rs_result), 2),
            round(float(rs_ci[0]), 2),
            round(float(rs_ci[1]), 2),
            # studies,
            # tissues,
            path,
        ]
    )
    return results



#### 3 Loop over different datasets (as they will have different rows and columns)

def comparisons_df_iterative(disdic,projectId):
    toAnalysis=[(key, value) for key, value in disdic.items() if value == projectId]
    schema = StructType(
        [
            StructField("comparison", StringType(), True),
            StructField("comparisonType", StringType(), True),
        ]
    )

    comparisons = spark.createDataFrame(toAnalysis, schema=schema)
    ### include all the columns as predictor

    predictions = spark.createDataFrame(
        data=[
            ("Phase4", "clinical"),
            ("Phase>=3", "clinical"),
            ("Phase>=2", "clinical"),
            ("Phase>=1", "clinical"),
            #("nPhase4", "clinical"),
            #("nPhase>=3", "clinical"),
            #("nPhase>=2", "clinical"),
            #("nPhase>=1", "clinical"),
            ("approved", "clinical"),

            # ("PhaseT", "clinical"),
        ]
    )
    return comparisons.join(predictions, how="full").collect()

full_data = spark.createDataFrame(
    data=[
        ("yes", "yes"),
        ("yes", "no"),
        ("no", "yes"),
        ("no", "no"),
    ],
    schema=StructType(
        [
            StructField("prediction", StringType(), True),
            StructField("comparison", StringType(), True),
        ]
    ),
)
print("created full_data and lists")

result = []
result_st = []
result_ci = []
array2 = []
listado = []
result_all = []
today_date = str(date.today())
variables_study = ["projectId", "biosampleName", "rightStudyType", "colocDoE"]

print("looping for variables_study")

for variable in variables_study:
    print("analysing",variable)
    #### build list of comparison and prediction columns
    rows=comparisons_df_iterative(disdic,variable)
    #### prepare aggregation depending on the variable problem
    window_spec = Window.partitionBy("targetId","diseaseId",variable).orderBy(F.col("pValueExponent").asc())
    #### take directionality from lowest p value
    bench2=benchmark.withColumn("agree_lowestPval", F.first("AgreeDrug").over(window_spec)
        ).groupBy("targetId", "diseaseId", "clinicalStudyId","clinicalPhase","maxClinPhase","approvedDrug").pivot(variable).agg(F.collect_set("agree_lowestPVal")
        ).withColumn(
        "Phase4",
        F.when(F.col("maxClinPhase") == 4, F.lit("yes")).otherwise(F.lit("no")),
        ).withColumn(
            "Phase>=3",
            F.when(F.col("maxClinPhase") >= 3, F.lit("yes")).otherwise(F.lit("no")),
        ).withColumn(
            "Phase>=2",
            F.when(F.col("maxClinPhase") >= 2, F.lit("yes")).otherwise(F.lit("no")),
        ).withColumn(
            "Phase>=1",
            F.when(F.col("maxClinPhase") >= 1, F.lit("yes")).otherwise(F.lit("no")),
        ).withColumn(
        "approved",
            F.when(F.col("approvedDrug")==1, F.lit("yes")).otherwise(F.lit("no")),
        )
    #### build columns yes/no for each distinct value in the column variable
    for x, value in [(key, val) for key, val in disdic.items() if val == variable]:
        print("building columns: ", x,"and",value)
        bench2 = bench2.withColumn(
            x, 
            F.when(F.array_contains(F.col(x),"yes"), F.lit("yes")).otherwise(F.lit("no"))
        )
    #### doing aggregations per 
    for row in rows:
        print("row:",row)
        results = aggregations_original(bench2, "propagated", listado, *row, today_date)
        result_all.append(results)
    
    schema = StructType(
    [
        StructField("group", StringType(), True),
        StructField("comparison", StringType(), True),
        StructField("phase", StringType(), True),
        StructField("oddsRatio", DoubleType(), True),
        StructField("pValue", DoubleType(), True),
        StructField("lowerInterval", DoubleType(), True),
        StructField("upperInterval", DoubleType(), True),
        StructField("total", StringType(), True),
        StructField("values", ArrayType(ArrayType(IntegerType())), True),
        StructField("relSuccess", DoubleType(), True),
        StructField("rsLower", DoubleType(), True),
        StructField("rsUpper", DoubleType(), True),
        StructField("path", StringType(), True),
    ]
)

# Convert list of lists to DataFrame
df = spreadSheetFormatter(spark.createDataFrame(result_all, schema=schema))
df.toPandas().to_csv(
    f"gs://ot-team/jroldan/analysis/{today_date}_credibleSetColocDoEanalysis_fixedIndex_fixedTotalNumber_CoherentThing.csv"
)

print("dataframe written \n Analysis finished")

In [6]:
#### build list of comparison and prediction columns
rows=comparisons_df_iterative(disdic,variables_study[2])[0]
#### prepare aggregation depending on the variable problem
window_spec = Window.partitionBy("targetId","diseaseId","rightStudyType").orderBy(F.col("pValueExponent").asc())
#### take directionality from lowest p value
bench2=benchmark.withColumn("agree_lowestPval", F.first("AgreeDrug").over(window_spec)
    ).groupBy("targetId", "diseaseId", "clinicalStudyId","clinicalPhase","maxClinPhase","approvedDrug").pivot("rightStudyType").agg(F.collect_set("agree_lowestPVal")
    ).withColumn(
    "Phase4",
    F.when(F.col("maxClinPhase") == 4, F.lit("yes")).otherwise(F.lit("no")),
    ).withColumn(
        "Phase>=3",
        F.when(F.col("maxClinPhase") >= 3, F.lit("yes")).otherwise(F.lit("no")),
    ).withColumn(
        "Phase>=2",
        F.when(F.col("maxClinPhase") >= 2, F.lit("yes")).otherwise(F.lit("no")),
    ).withColumn(
        "Phase>=1",
        F.when(F.col("maxClinPhase") >= 1, F.lit("yes")).otherwise(F.lit("no")),
    ).withColumn(
    "approved",
        F.when(F.col("approvedDrug")==1, F.lit("yes")).otherwise(F.lit("no")),
    )
#### build columns yes/no for each distinct value in the column variable


NameError: name 'comparisons_df_iterative' is not defined

In [86]:
for x, value in [(key, val) for key, val in disdic.items() if val == "rightStudyType"]:
    print(x,value)

eqtl rightStudyType
sqtl rightStudyType
tuqtl rightStudyType
sceqtl rightStudyType
pqtl rightStudyType
sctuqtl rightStudyType
scsqtl rightStudyType


In [90]:
x="eqtl"
bench2.withColumn(
        x, 
        F.when(F.array_contains(F.array(F.col(x)),"yes"), F.lit("yes")).otherwise(F.lit("no"))
    ).groupBy(x).pivot("phase4").count().show()

+----+----+---+
|eqtl|  no|yes|
+----+----+---+
|  no|1916|825|
| yes|1293|379|
+----+----+---+



In [94]:
x="pqtl"
bench2.withColumn(
        x, 
        F.when(F.array_contains(F.array(F.col(x)),"yes"), F.lit("yes")).otherwise(F.lit("no"))
    ).groupBy(x).pivot("phase4").count().show()

+----+----+----+
|pqtl|  no| yes|
+----+----+----+
|  no|2944|1141|
| yes| 265|  63|
+----+----+----+



In [93]:
bench2.withColumn(
        x, 
        F.when(F.array_contains(F.array(F.col(x)),"yes"), F.lit("yes")).otherwise(F.lit("no"))
    ).sort("targetId","diseaseId",F.col("clinicalPhase").asc()).show()

+---------------+-----------+---------------+-------------+------------+------------+----+----+------+------+-------+----+-----+------+--------+--------+--------+--------+
|       targetId|  diseaseId|clinicalStudyId|clinicalPhase|maxClinPhase|approvedDrug|eqtl|pqtl|sceqtl|scsqtl|sctuqtl|sqtl|tuqtl|Phase4|Phase>=3|Phase>=2|Phase>=1|approved|
+---------------+-----------+---------------+-------------+------------+------------+----+----+------+------+-------+----+-----+------+--------+--------+--------+--------+
|ENSG00000006071|EFO_0000400|    NCT01489644|          1.0|         1.0|           1|  no|  no|    no|    no|     no|  no|   no|    no|      no|      no|     yes|     yes|
|ENSG00000006071|EFO_0000400|    NCT02954822|          1.0|         1.0|           1|  no|  no|    no|    no|     no|  no|   no|    no|      no|      no|     yes|     yes|
|ENSG00000006071|EFO_0000400|    NCT00959101|          1.0|         1.0|           1|  no|  no|    no|    no|     no|  no|   no|    no|     

In [85]:
for x, value in [(key, val) for key, val in disdic.items() if val == "rightStudyType"]:
    print("building columns: ", x,"and",value)
    bench2 = bench2.withColumn(
        x, 
        F.when(F.array_contains(F.col(x),"yes"), F.lit("yes")).otherwise(F.lit("no"))
    ).show()

building columns:  eqtl and rightStudyType


AnalysisException: cannot resolve 'array_contains(eqtl, 'yes')' due to data type mismatch: Input to function array_contains should have been array followed by a value with same element type, but it's [string, string].;
'Project [targetId#2095238, diseaseId#2095387, clinicalStudyId#2102030, clinicalPhase#2096480, maxClinPhase#2098565, approvedDrug#2098689, CASE WHEN array_contains(eqtl#4471932, yes) THEN yes ELSE no END AS eqtl#4472065, pqtl#4472008, sceqtl#4471989, scsqtl#4472046, sctuqtl#4472027, sqtl#4471951, tuqtl#4471970, Phase4#4471847, Phase>=3#4471862, Phase>=2#4471878, Phase>=1#4471895, approved#4471913]
+- Project [targetId#2095238, diseaseId#2095387, clinicalStudyId#2102030, clinicalPhase#2096480, maxClinPhase#2098565, approvedDrug#2098689, eqtl#4471932, pqtl#4472008, sceqtl#4471989, CASE WHEN array_contains(scsqtl#4471827, yes) THEN yes ELSE no END AS scsqtl#4472046, sctuqtl#4472027, sqtl#4471951, tuqtl#4471970, Phase4#4471847, Phase>=3#4471862, Phase>=2#4471878, Phase>=1#44

In [ ]:
benchmark.groupBy("")

### Take examples of the numbers that we observe in the statistics 

In [22]:
### include drugId and studyId becuase there are some studies where there are multiple drugs. 
# Everywthing will become coherent in chembl assoc since this
chemblAssoc = (

    discrepancifier(
        assessment.filter(
            (F.col("datasourceId") == "chembl")
            & (F.col("homogenized") != "noEvaluable")
        )
        .join(drugApproved.filter(F.col("isApproved")==1), on=["targetId","diseaseId","drugId"], how="left")
        .withColumn(
            "maxClinPhase",
            F.max("clinicalPhase").over(Window.partitionBy("targetId", "diseaseId")),
        )
        .withColumn(
        "approvedDrug",
        F.max(F.col("isApproved")).over(Window.partitionBy("targetId", "diseaseId")),
        )
        .groupBy("targetId", "diseaseId","drugId", "studyId","clinicalPhase","maxClinPhase","approvedDrug")
        .pivot("homogenized")
        .count()
    )
    .filter(F.col("coherencyDiagonal") == "coherent")
    .drop(
        "coherencyDiagonal", "coherencyOneCell", "noEvaluable", "GoF_risk", "LoF_risk"
    )
    .withColumnRenamed("GoF_protect", "drugGoF_protect")
    .withColumnRenamed("LoF_protect", "drugLoF_protect")
)

In [21]:
new_chemblAssoc.show()

+---------------+-------------+-------------+-----------+-------------+------------+---------------+---------------+
|       targetId|    diseaseId|       drugId|    studyId|clinicalPhase|maxClinPhase|drugGoF_protect|drugLoF_protect|
+---------------+-------------+-------------+-----------+-------------+------------+---------------+---------------+
|ENSG00000007314|  EFO_0000555|     CHEMBL79|NCT00108446|          2.0|         2.0|           null|              1|
|ENSG00000007314|  EFO_0000555|   CHEMBL1404|NCT02239926|          2.0|         2.0|           null|              1|
|ENSG00000007314|  EFO_0000555|    CHEMBL558|NCT01717404|          1.0|         2.0|           null|              1|
|ENSG00000007314|  EFO_0003102|    CHEMBL569|NCT02099240|          0.5|         0.5|           null|              1|
|ENSG00000007314|  EFO_0003894|     CHEMBL79|NCT01807455|          4.0|         4.0|           null|              1|
|ENSG00000007314|  EFO_0004699| CHEMBL220492|NCT00245583|       

In [16]:
discrepancifier(
        assessment.filter(
            (F.col("datasourceId") == "chembl")
            & (F.col("homogenized") != "noEvaluable")
        )
        .withColumn(
            "maxClinPhase",
            F.max("clinicalPhase").over(Window.partitionBy("targetId", "diseaseId")),
        )
        .groupBy("targetId", "diseaseId", "studyId","clinicalPhase","maxClinPhase")
        .pivot("homogenized")
        .count()
    ).groupBy("coherencyDiagonal").count().show()

25/01/08 12:01:01 WARN CacheManager: Asked to cache already cached data.
25/01/08 12:01:01 WARN CacheManager: Asked to cache already cached data.
25/01/08 12:01:01 WARN CacheManager: Asked to cache already cached data.


+-----------------+------+
|coherencyDiagonal| count|
+-----------------+------+
|           dispar|   760|
|         coherent|445772|
+-----------------+------+



In [18]:
discrepancifier(
        assessment.filter(
            (F.col("datasourceId") == "chembl")
            & (F.col("homogenized") != "noEvaluable")
        )
        .withColumn(
            "maxClinPhase",
            F.max("clinicalPhase").over(Window.partitionBy("targetId", "diseaseId")),
        )
        .groupBy("targetId", "diseaseId", "drugId","clinicalPhase","maxClinPhase")
        .pivot("homogenized")
        .count()
    ).groupBy("coherencyDiagonal").count().show()

+-----------------+------+
|coherencyDiagonal| count|
+-----------------+------+
|         coherent|197723|
+-----------------+------+



In [17]:
discrepancifier(
        assessment.filter(
            (F.col("datasourceId") == "chembl")
            & (F.col("homogenized") != "noEvaluable")
        )
        .withColumn(
            "maxClinPhase",
            F.max("clinicalPhase").over(Window.partitionBy("targetId", "diseaseId")),
        )
        .groupBy("targetId", "diseaseId","drugId", "studyId","clinicalPhase","maxClinPhase")
        .pivot("homogenized")
        .count()
    ).groupBy("coherencyDiagonal").count().show()

+-----------------+------+
|coherencyDiagonal| count|
+-----------------+------+
|         coherent|538206|
+-----------------+------+



In [4]:
assessment.filter(
            (F.col("datasourceId") == "chembl")
            & (F.col("homogenized") != "noEvaluable")
        ).show()

+------------+---------------+-------------+-------------------+--------+----------+------+-----------+----+---------------------------+---------------------------+---------------------------------+--------------------------------+-----------------+-------------+-------------+----------+--------------------+------------------+--------+-------------+---------------------+--------------+-----------------+--------+----------------+---------------+----------+--------+-------------------+----------+----------------+--------------------+-------------------+-------------------------+-------------------------------------+-------------------------------------+--------------+-------------+------------+-------------------+-----------------+------------------------+-----------------------+-----------------------------+---------------------+----------+----------------------------+-------------------+--------------+---------+--------------------------------+--------------------------------+---------

### Runing analysis using drugId

In [ ]:
##### Run the analysis using Drug Id


#### 08.01.2025
#### ALL PHASES 
from array import ArrayType
from functions import (
    relative_success,
    spreadSheetFormatter,
    discrepancifier,
    temporary_directionOfEffect,
)
from stoppedTrials import terminated_td
from DoEAssessment import directionOfEffect
from membraneTargets import target_membrane
from pyspark.sql import SparkSession, Window
import pyspark.sql.functions as F
from datetime import datetime
from datetime import date
from pyspark.sql.types import StructType, StructField, StringType, IntegerType
from pyspark.sql.types import (
    StructType,
    StructField,
    DoubleType,
    DecimalType,
    StringType,
    FloatType,
)
import pandas as pd

spark = SparkSession.builder.getOrCreate()

path = "gs://open-targets-pre-data-releases/24.12-uo_test-3/output/etl/parquet/"

target = spark.read.parquet(f"{path}targets/")

diseases = spark.read.parquet(f"{path}diseases/")

evidences = spark.read.parquet(f"{path}evidence")

credible = spark.read.parquet(f"{path}credibleSet")

### index with new fix" "gs://ot-team/irene/gentropy/study_index_2412_fixed"
index = spark.read.parquet(f"gs://ot-team/irene/gentropy/study_index_2412_fixed")

new = spark.read.parquet(f"{path}colocalisation/coloc")

variantIndex = spark.read.parquet(f"{path}variantIndex")

biosample = spark.read.parquet(f"{path}biosample")


#### Fixing scXQTL as XQTLs:
## code provided by @ireneisdoomed
pd.DataFrame.iteritems = pd.DataFrame.items

raw_studies_metadata_schema: StructType = StructType(
        [
            StructField("study_id", StringType(), True),
            StructField("dataset_id", StringType(), True),
            StructField("study_label", StringType(), True),
            StructField("sample_group", StringType(), True),
            StructField("tissue_id", StringType(), True),
            StructField("tissue_label", StringType(), True),
            StructField("condition_label", StringType(), True),
            StructField("sample_size", IntegerType(), True),
            StructField("quant_method", StringType(), True),
            StructField("pmid", StringType(), True),
            StructField("study_type", StringType(), True),
        ]
    )
raw_studies_metadata_path = "https://raw.githubusercontent.com/eQTL-Catalogue/eQTL-Catalogue-resources/fe3c4b4ed911b3a184271a6aadcd8c8769a66aba/data_tables/dataset_metadata.tsv"

study_table = spark.createDataFrame(
            pd.read_csv(raw_studies_metadata_path, sep="\t"),
            schema=raw_studies_metadata_schema,
        )

#index = spark.read.parquet("gs://open-targets-pre-data-releases/24.12-uo_test-3/output/genetics/parquet/study_index")

study_index_w_correct_type = (
    study_table.select(
        F.concat_ws(
            "_",
            F.col("study_label"),
            F.col("quant_method"),
            F.col("sample_group"),
        ).alias("extracted_column"),
        "study_type",
    )
    .join(
        index
        # Get eQTL Catalogue studies
        .filter(F.col("studyType") != "gwas")
        .filter(~F.col("studyId").startswith("UKB_PPP"))
        # Remove measured trait
        .withColumn(
            "extracted_column",
            F.regexp_replace(F.col("studyId"), r"(_ENS.*|_ILMN.*|_X.*|_[0-9]+:.*)", ""),
        )
        .withColumn(
            "extracted_column",
            # After the previous cleanup, there are multiple traits from the same publication starting with the gene symbol that need to be removed (e.g. `Sun_2018_aptamer_plasma_ANXA2.4961.17.1..1`)
            F.when(
                F.col("extracted_column").startswith("Sun_2018_aptamer_plasma"),
                F.lit("Sun_2018_aptamer_plasma"),
            ).otherwise(F.col("extracted_column")),
        ),
        on="extracted_column",
        how="right",
    )
    .persist()
)

fixed = (
    study_index_w_correct_type.withColumn(
        "toFix",
        F.when(
            (F.col("study_type") != "single-cell")
            & (F.col("studyType").startswith("sc")),
            F.lit(True),
        ).otherwise(F.lit(False)),
    )
    # Remove the substring "sc" from the studyType column
    .withColumn(
        "newStudyType",
        F.when(
            F.col("toFix"), F.regexp_replace(F.col("studyType"), r"sc", "")
        ).otherwise(F.col("studyType")),
    )
    .drop("toFix", "extracted_column", "study_type")
).persist()
all_studies = index.join(
    fixed.selectExpr("studyId", "newStudyType"), on="studyId", how="left"
).persist()
fixedIndex = all_studies.withColumn(
    "studyType",
    F.when(F.col("newStudyType").isNotNull(), F.col("newStudyType")).otherwise(
        F.col("studyType")
    ),
).drop("newStudyType")
#### fixed  

newColoc = (
    new.join(
        credible.selectExpr(  #### studyLocusId from credible set to uncover the codified variants on left side
            "studyLocusId as leftStudyLocusId",
            "StudyId as leftStudyId",
            "variantId as leftVariantId",
            "studyType as credibleLeftStudyType",
        ),
        on="leftStudyLocusId",
        how="left",
    )
    .join(
        credible.selectExpr(  #### studyLocusId from credible set to uncover the codified variants on right side
            "studyLocusId as rightStudyLocusId",
            "studyId as rightStudyId",
            "variantId as rightVariantId",
            "studyType as credibleRightStudyType",
        ),
        on="rightStudyLocusId",
        how="left",
    )
    .join(
        fixedIndex.selectExpr(  ### bring modulated target on right side (QTL study)
            "studyId as rightStudyId", "geneId", "projectId", "studyType as indexStudyType", "condition", "biosampleId"
        ),
        on="rightStudyId",
        how="left",
    )
    .persist()
)
# remove columns without content (only null values on them)
df = evidences.filter((F.col("datasourceId") == "gwas_credible_sets"))

# Use an aggregation to determine non-null columns
non_null_counts = df.select(
    *[F.sum(F.col(col).isNotNull().cast("int")).alias(col) for col in df.columns]
)

# Collect the counts for each column
non_null_columns = [
    row[0] for row in non_null_counts.collect()[0].asDict().items() if row[1] > 0
]

# Select only the non-null columns
filtered_df = df.select(*non_null_columns).persist()

## bring studyId, variantId, beta from Gwas and pValue
gwasComplete = filtered_df.join(
    credible.selectExpr(
        "studyLocusId", "studyId", "variantId", "beta as betaGwas", "pValueExponent"
    ),
    on="studyLocusId",
    how="left",
).persist()

resolvedColoc = (
    (
        newColoc.withColumnRenamed("geneId", "targetId")
        .join(
            gwasComplete.withColumnRenamed("studyLocusId", "leftStudyLocusId"),
            on=["leftStudyLocusId", "targetId"],
            how="inner",
        )
        .join(  ### propagated using parent terms
            diseases.selectExpr(
                "id as diseaseId", "name", "parents", "therapeuticAreas"
            ),
            on="diseaseId",
            how="left",
        )
        .withColumn(
            "diseaseId",
            F.explode_outer(F.concat(F.array(F.col("diseaseId")), F.col("parents"))),
        )
        .drop("parents", "oldDiseaseId")
    )
    .withColumn(
        "colocDoE",
        F.when(
            F.col("rightStudyType").isin(
                ["eqtl", "pqtl", "tuqtl", "sceqtl", "sctuqtl"]
            ),
            F.when(
                (F.col("betaGwas") > 0) & (F.col("betaRatioSignAverage") > 0),
                F.lit("GoF_risk"),
            )
            .when(
                (F.col("betaGwas") > 0) & (F.col("betaRatioSignAverage") < 0),
                F.lit("LoF_risk"),
            )
            .when(
                (F.col("betaGwas") < 0) & (F.col("betaRatioSignAverage") > 0),
                F.lit("LoF_protect"),
            )
            .when(
                (F.col("betaGwas") < 0) & (F.col("betaRatioSignAverage") < 0),
                F.lit("GoF_protect"),
            ),
        ).when(
            F.col("rightStudyType").isin(
                ["sqtl", "scsqtl"]
            ),  ### opposite directionality than sqtl
            F.when(
                (F.col("betaGwas") > 0) & (F.col("betaRatioSignAverage") > 0),
                F.lit("LoF_risk"),
            )
            .when(
                (F.col("betaGwas") > 0) & (F.col("betaRatioSignAverage") < 0),
                F.lit("GoF_risk"),
            )
            .when(
                (F.col("betaGwas") < 0) & (F.col("betaRatioSignAverage") > 0),
                F.lit("GoF_protect"),
            )
            .when(
                (F.col("betaGwas") < 0) & (F.col("betaRatioSignAverage") < 0),
                F.lit("LoF_protect"),
            ),
        ),
    )
    .persist()
)

path = "gs://open-targets-pre-data-releases/24.12-uo_test-3/output/etl/parquet/"

datasource_filter = [
    "ot_genetics_portal",
    "gwas_credible_sets",
    "gene_burden",
    "eva",
    "eva_somatic",
    "gene2phenotype",
    "orphanet",
    "cancer_gene_census",
    "intogen",
    "impc",
    "chembl",
]

assessment, evidences, actionType, oncolabel = temporary_directionOfEffect(
    path, datasource_filter
)

drugApproved=spark.read.parquet("gs://ot-team/irene/l2g/validation/chembl_w_flags").drop("clinicalTrialId","isComplex"
).withColumn("isApproved", F.when(F.col("isApproved")=="true", F.lit(1)).otherwise(F.lit(0))).distinct()

### include drugId and studyId becuase there are some studies where there are multiple drugs. 
# Everywthing will become coherent in chembl assoc since this
### 1nd version of 
analysis_chembl_indication = (
    discrepancifier(
        assessment.filter(
            (F.col("datasourceId") == "chembl")
            & (F.col("homogenized") != "noEvaluable")
        )
        .join(drugApproved.filter(F.col("isApproved")==1), on=["targetId","diseaseId","drugId"], how="left")
        .withColumn(
            "maxClinPhase", ### no longer maxclinphase for T-D
            F.max("clinicalPhase").over(Window.partitionBy("targetId", "diseaseId","drugId", "studyId","clinicalPhase")),
        )
        .withColumn(
        "approvedDrug",
        F.max(F.col("isApproved")).over(Window.partitionBy("targetId", "diseaseId")),
        )
        ## do not consider drugId so when there are two drugs with diferent MoA at the same clinical phase we will not include it (coherency==coherent)
        .groupBy("targetId", "diseaseId", "studyId","drugId","clinicalPhase","maxClinPhase","approvedDrug")  ## we can remove drugID
        .pivot("homogenized")
        .count()
    )
    .withColumnRenamed("studyId","clinicalStudyId")
    .filter(F.col("coherencyDiagonal") == "coherent")
    .drop(
        "coherencyDiagonal", 
        "coherencyOneCell", 
        "noEvaluable", 
        "GoF_risk", 
        "LoF_risk"
    )
    .withColumnRenamed("GoF_protect", "drugGoF_protect")
    .withColumnRenamed("LoF_protect", "drugLoF_protect")
)

In [ ]:
benchmark = (
    (
        resolvedColoc.filter(F.col("betaGwas") < 0)
        .join(  ### select just GWAS giving protection
            analysis_chembl_indication, on=["targetId", "diseaseId"], how="inner"
        )
        .withColumn(
            "AgreeDrug",
            F.when(
                (F.col("drugGoF_protect").isNotNull())
                & (F.col("colocDoE") == "GoF_protect"),
                F.lit("yes"),
            )
            .when(
                (F.col("drugLoF_protect").isNotNull())
                & (F.col("colocDoE") == "LoF_protect"),
                F.lit("yes"),
            )
            .otherwise(F.lit("no")),
        )
    )
    .filter(F.col("name") != "COVID-19")  #### remove COVID-19 associations
).join(biosample.select("biosampleId", "biosampleName"), on="biosampleId", how="left")


In [ ]:
####2 Define agregation function
import pandas as pd
import numpy as np
from scipy.stats import fisher_exact
from scipy.stats.contingency import odds_ratio
from pyspark.sql.types import *

def convertTuple(tup):
    st = ",".join(map(str, tup))
    return st

#####3 run in a function
def aggregations_original(
    df,
    data,
    listado,
    comparisonColumn,
    comparisonType,
    predictionColumn,
    predictionType,
    today_date,
):
    wComparison = Window.partitionBy(comparisonColumn)
    wPrediction = Window.partitionBy(predictionColumn)
    wPredictionComparison = Window.partitionBy(comparisonColumn, predictionColumn)
    results = []
    # uniqIds = df.select("targetId", "diseaseId").distinct().count()
    out = (
        df.withColumn("comparisonType", F.lit(comparisonType))
        .withColumn("dataset", F.lit(data))
        .withColumn("predictionType", F.lit(predictionType))
        # .withColumn("total", F.lit(uniqIds))
        .withColumn("a", F.count("targetId").over(wPredictionComparison))
        .withColumn("comparisonColumn", F.lit(comparisonColumn))
        .withColumn("predictionColumnValue", F.lit(predictionColumn))
        .withColumn(
            "predictionTotal",
            F.count("targetId").over(wPrediction),
        )
        .withColumn(
            "comparisonTotal",
            F.count("targetId").over(wComparison),
        )
        .select(
            F.col(predictionColumn).alias("prediction"),
            F.col(comparisonColumn).alias("comparison"),
            "dataset",
            "comparisonColumn",
            "predictionColumnValue",
            "comparisonType",
            "predictionType",
            "a",
            "predictionTotal",
            "comparisonTotal",
        )
        .filter(F.col("prediction").isNotNull())
        .filter(F.col("comparison").isNotNull())
        .distinct()
    )
    '''
    out.write.mode("overwrite").parquet(
        "gs://ot-team/jroldan/"
        + str(
            today_date
            + "_"
            + "analysis/"
            + data
            # + "_propagated"
            + "/"
            + comparisonColumn
            + "_"
            + comparisonType
            + "_"
            + predictionColumn
            + ".parquet"
        )
    )
    '''
    
    listado.append(
        "gs://ot-team/jroldan/"
        + str(
            today_date
            + "_"
            + "analysis/"
            + data
            # + "_propagated"
            + "/"
            + comparisonColumn
            + "_"
            + comparisonType
            + "_"
            + predictionColumn
            + ".parquet"
        )
    )
    path = "gs://ot-team/jroldan/" + str(
        today_date
        + "_"
        + "analysis/"
        + data
        # + "_propagated"
        + "/"
        + comparisonColumn
        + "_"
        + comparisonType
        + "_"
        + predictionColumn
        + ".parquet"
    )
    print(path)
    ### making analysis
    array1 = np.delete(
        out.join(full_data, on=["prediction", "comparison"], how="outer")
        .groupBy("comparison")
        .pivot("prediction")
        .agg(F.first("a"))
        .sort(F.col("comparison").desc())
        .select("comparison", "yes", "no")
        .fillna(0)
        .toPandas()
        .to_numpy(),
        [0],
        1,
    )
    total = np.sum(array1)
    res_npPhaseX = np.array(array1, dtype=int)
    resX = convertTuple(fisher_exact(res_npPhaseX, alternative="two-sided"))
    resx_CI = convertTuple(
        odds_ratio(res_npPhaseX).confidence_interval(confidence_level=0.95)
    )

    result_st.append(resX)
    result_ci.append(resx_CI)
    (rs_result, rs_ci) = relative_success(array1)
    results.extend(
        [
            comparisonType,
            comparisonColumn,
            predictionColumn,
            round(float(resX.split(",")[0]), 2),
            float(resX.split(",")[1]),
            round(float(resx_CI.split(",")[0]), 2),
            round(float(resx_CI.split(",")[1]), 2),
            str(total),
            np.array(res_npPhaseX).tolist(),
            round(float(rs_result), 2),
            round(float(rs_ci[0]), 2),
            round(float(rs_ci[1]), 2),
            # studies,
            # tissues,
            path,
        ]
    )
    return results



#### 3 Loop over different datasets (as they will have different rows and columns)

def comparisons_df_iterative(disdic,projectId):
    toAnalysis=[(key, value) for key, value in disdic.items() if value == projectId]
    schema = StructType(
        [
            StructField("comparison", StringType(), True),
            StructField("comparisonType", StringType(), True),
        ]
    )

    comparisons = spark.createDataFrame(toAnalysis, schema=schema)
    ### include all the columns as predictor

    predictions = spark.createDataFrame(
        data=[
            ("Phase4", "clinical"),
            ("Phase>=3", "clinical"),
            ("Phase>=2", "clinical"),
            ("Phase>=1", "clinical"),
            #("nPhase4", "clinical"),
            #("nPhase>=3", "clinical"),
            #("nPhase>=2", "clinical"),
            #("nPhase>=1", "clinical"),
            ("approved", "clinical"),

            # ("PhaseT", "clinical"),
        ]
    )
    return comparisons.join(predictions, how="full").collect()

full_data = spark.createDataFrame(
    data=[
        ("yes", "yes"),
        ("yes", "no"),
        ("no", "yes"),
        ("no", "no"),
    ],
    schema=StructType(
        [
            StructField("prediction", StringType(), True),
            StructField("comparison", StringType(), True),
        ]
    ),
)
print("created full_data and lists")

result = []
result_st = []
result_ci = []
array2 = []
listado = []
result_all = []
today_date = str(date.today())
variables_study = ["projectId", "biosampleName", "rightStudyType", "colocDoE"]

print("looping for variables_study")

for variable in variables_study:
    print("analysing",variable)
    #### build list of comparison and prediction columns
    rows=comparisons_df_iterative(disdic,variable)
    #### prepare aggregation depending on the variable problem
    window_spec = Window.partitionBy("targetId","diseaseId",variable).orderBy(F.col("pValueExponent").asc())
    #### take directionality from lowest p value
    bench2=benchmark.withColumn("agree_lowestPval", F.first("AgreeDrug").over(window_spec)
        ).groupBy("targetId", "diseaseId", "clinicalStudyId","clinicalPhase","maxClinPhase","approvedDrug").pivot(variable).agg(F.collect_set("agree_lowestPVal")
        ).withColumn(
        "Phase4",
        F.when(F.col("maxClinPhase") == 4, F.lit("yes")).otherwise(F.lit("no")),
        ).withColumn(
            "Phase>=3",
            F.when(F.col("maxClinPhase") >= 3, F.lit("yes")).otherwise(F.lit("no")),
        ).withColumn(
            "Phase>=2",
            F.when(F.col("maxClinPhase") >= 2, F.lit("yes")).otherwise(F.lit("no")),
        ).withColumn(
            "Phase>=1",
            F.when(F.col("maxClinPhase") >= 1, F.lit("yes")).otherwise(F.lit("no")),
        ).withColumn(
        "approved",
            F.when(F.col("approvedDrug")==1, F.lit("yes")).otherwise(F.lit("no")),
        )
    #### build columns yes/no for each distinct value in the column variable
    for x, value in [(key, val) for key, val in disdic.items() if val == variable]:
        print("building columns: ", x,"and",value)
        bench2 = bench2.withColumn(
            x, 
            F.when(F.array_contains(F.col(x),"yes"), F.lit("yes")).otherwise(F.lit("no"))
        )
    #### doing aggregations per 
    for row in rows:
        print("row:",row)
        results = aggregations_original(bench2, "propagated", listado, *row, today_date)
        result_all.append(results)
    
    schema = StructType(
    [
        StructField("group", StringType(), True),
        StructField("comparison", StringType(), True),
        StructField("phase", StringType(), True),
        StructField("oddsRatio", DoubleType(), True),
        StructField("pValue", DoubleType(), True),
        StructField("lowerInterval", DoubleType(), True),
        StructField("upperInterval", DoubleType(), True),
        StructField("total", StringType(), True),
        StructField("values", ArrayType(ArrayType(IntegerType())), True),
        StructField("relSuccess", DoubleType(), True),
        StructField("rsLower", DoubleType(), True),
        StructField("rsUpper", DoubleType(), True),
        StructField("path", StringType(), True),
    ]
)

# Convert list of lists to DataFrame
df = spreadSheetFormatter(spark.createDataFrame(result_all, schema=schema))
df.toPandas().to_csv(
    f"gs://ot-team/jroldan/analysis/{today_date}_credibleSetColocDoEanalysis_fixedIndex_fixedTotalNumber_CoherentThing.csv"
)

print("dataframe written \n Analysis finished")